In [ ]:
## Полезная штука, если данных много и в аутпуте всё не помещается.
## В текущей директории создаётся тхт файл с полным содержимым.
#
#
#series_for_output = pd.Series(booking_df['negative_review'].value_counts().reset_index()['negative_review'].sort_values().str.strip().str.lower().unique())
#
## Сюда нужно поместить то, что хотим полностью увидеть.
#output_data = series_for_output[series_for_output.apply(lambda x: len(x)) < 14]

## Если аутпут - сериес:
#with open('output_neg_rev.txt', 'w') as f:
#    output_data.to_csv('output_neg_rev.txt', sep='\t', index=False)

## Если аутпут - списко:
#with open('output.txt', 'w') as f:
#    for item in output_data:
#        f.write("%s\n" % item)

In [1]:
# А эта строчка позволит посмотреть, какие в памяти имеются глобальные переменные:
# [var for var in globals() if not var.startswith('_')]

<h2 id='contents'>Оглавляшка:</h2>

1. [Функция-аналог .info() - my_info()](#myinfo)
2. Графические функции:
    * [Функция рисования 3-х графиков: гистограммы, коробки и Q-Q в одном флаконе.](#histboxqq)(для массива чисел)
    * [Beeswarm + hist для двух переменных](#beeswarmhist)
    * [Функция рисования графика корреляции.](#mycorr)
    * [Функция декорирования оси Х столбчатой диаграммы.](#decoratexlabels)(крутая штука)
    * [Функция рисования порогов вероятности для метрик recall, precision и f1_score](#recprecfthreshplot)
    * [Функция рисования нахождения оптимального К для алгоритма K-MEANS с помощью inertia и silhouette](#kmeansinertiasilhouette)
    * [Функция построения кривой обучения](#educurvebuildfunc) learning_curve()
    * [Функция построения precision-recall кривой с меткой наилучшего порога](#prcurve) (PR-кривая)
3. [Функция, создающая словарь для .fillna() заполнения NaN.](#fillnadict)
4. [Функция отлова выбросов по методу Тьюки.](#outlierstukie)
5. [Функция проверки на нормальность Шапиро-Уилка.](#shapirowilkfunc)
6. [Функция проверки равенства дисперсий от Левена.](#levenefunc)
7. Функция для работы с ["строками", "внутри", "списка тэгов", "1 Adult", "2 Adults"]
    * [ОСНОВНАЯ](#maintagsworkfunc)
    * [Второстепенная](#retagsworkfunc)
8. Функция для работы с текстом. Когда столбец состоит только из элементов str
    * [Основная функция с алгоритмом Вейдера](#vadersentanalmain)
    * [Второстепенная функция с алгоритмом Вейдера](#vadersentanalsec)
    * [Функция с алгоритмом Роберты на основе отзывов в твиттере](#robertapolarity)
9. Функции для подбора гиперпараметров.
    * [Optuna](#optunafunc)
10. Математика
    * [Исследование функции для одной переменной](#func_research) сырая
    * [Исследование функции для двух, трёх переменных](#func_research_xyz) сырая
    * [Интерактивный график градиентного спуска для функции одной переменной.](#gradfxinteractive)
11. География
    * [Функция, которая вычисляет расстояние между двумя точками lattitude и longitude.](#latlongdist)
12. [Функция для тренировки нескольких моделей, используя несколько алгоритмов ML за 1 заход](#multiplemodelstraining)
13. [Функция для генерации комбинаций элементов входящего списка](#combinationsgenerator) (классный комбинатор элементов списка)
14. [Функция для тестирования силы столбца, относительно целевого. f_classif + chi2](#fclassifchitester)
15. [Группировщик с применением ко всему датафрейму](#specialgrouper)
16. [Функция для подсчёта любых метрик на выбор, как для и просто тест выборки, так и трейн + тест выборок](#printmetrics) вывод любых метрик
17. Тестовые функции:
    * [Функция для удаления сильно скоррелированных столбцов. (тестовая)](#highcorreraser)
    * [Функция для автоматического поиска сильных столбцов относительно целевого столбца (тестовая)](#strongcolssearcher)
    * ТЕСТОВАЯ [Функция для определения положительных и отрицательных слов в предложении](#goodbadwordssearcher) (супер сырая)

Пример работы tqdm

In [1]:
from tqdm.notebook import tqdm
import time

list_for_tqdm = [i for i in range(100)]

total_progress = 100
with tqdm(total=total_progress, desc=f"Cell progress: ", bar_format='{desc}: {percentage:.0f}%|{bar}| {n:.0f}/{total_fmt} [Elapsed: {elapsed} m/s]') as pbar:
    cumulative_progress = 0 # Чтобы прогресс бар работал, как надо и завершался всегда на 100.
##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############
# сюда нужно вставить свой цикл.
    for i, word in enumerate(list_for_tqdm):
        time.sleep(0.1)
        
##########vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv##############        
        # Рассчитываем размер шага на основе оставшегося прогресса и оставшихся итераций.
        remaining_iterations = len(list_for_tqdm) - i # вот тут пригодился индекс i
        remaining_progress = total_progress - cumulative_progress
        step_size = remaining_progress / remaining_iterations
        
        # Обновляем общий прогресс.
        cumulative_progress += step_size
        pbar.update(step_size)

Cell progress: : 0%|          | 0/100 [Elapsed: 00:00 m/s]

<h2 id='myinfo'>Функция печатающая аналог <span style='color:orange'>.info()</span> в расширенном варианте.</h2>

In [ ]:
# My info.
def my_info(df, catmaxnum=None):
    """Info function to show basic df indicators

    Args:
        df (pd.Dataframe): any df.
        catmaxnum (int): the minimum threshold(rus: порог) after which a column will be considered a category. Defaults to None.

    Returns:
        _type_: HTML object
    """
    import pandas as pd
    import numpy as np
    from IPython.display import HTML
    
    df_copy = df.copy() # it needs here.
    
    # Loop and condition for moment, when there are unhashable types in our column. (list, dict, set)
    for i in df_copy.columns:
        if any(isinstance(cell, (list, set, dict)) for cell in df_copy[i]):
            df_copy[i] = df_copy[i].astype(str)
                
    func_df = pd.DataFrame({
        
        'column': df_copy.columns,
        'num of unique vals': df_copy.nunique(),
        'type': [str(df_copy.dtypes[i]) for i in df_copy.columns],
        'mode': [round(df_copy[i].mode()[0], 2) if pd.api.types.is_numeric_dtype(df_copy[i].dtype) else df_copy[i].mode()[0] for i in df_copy.columns],
        'number of entries': len(df_copy),
        'NaN vals': df_copy.isna().sum(),
        'number of dublics': df_copy.duplicated().sum(),
        'describe': [f"{df_copy[i].describe().reindex(['min', 'max', 'mean', 'std']).round(2).to_string()}" if pd.api.types.is_numeric_dtype(df_copy[i].dtype) else 'see type column' for i in df_copy.columns]
        
    }).sort_values(by=['num of unique vals', 'column'], ascending=[True, True]).reset_index(drop=True)
    
    if catmaxnum:
        func_df.insert(3, 'classification', ['category' if i < catmaxnum else 'numeric' for i in func_df['num of unique vals']])
    
    if len(str(df_copy.memory_usage(deep=True).sum())) > 6:
        print(f"memory usage: {str(round(df_copy.memory_usage(deep=True).sum() / 1e6, 1)) + ' MB'}")
    else:
        print(f"memory usage: {str(round(df_copy.memory_usage(deep=True).sum() / 1e3, 1)) + ' KB'}")
    
    # Add CSS style to left-align the "describe" column
    html_output = func_df.to_html().replace('\\n', '<br>')
    html_output = html_output.replace('<td>min', '<td style="text-align: left;">min...:')
    html_output = html_output.replace('<br>max', '<br style="text-align: left;">max..:')
    html_output = html_output.replace('<br>mean', '<br style="text-align: left;">mean:')
    html_output = html_output.replace('<br>std', '<br style="text-align: left;">std....:')
    
    return HTML(html_output)

[to contents](#contents)

<h2 id='outlierstukie'>Функция отлова выбросов по методу Тьюки</h2>

In [ ]:
import pandas as pd
import numpy as np

# Tukie outliers finder function.
def outliers_iqr_mode_log(data, feature=None, log_scale=False, left=1.5, right=1.5):
    
    func_data = data.copy()
    
    if isinstance(func_data, pd.DataFrame):    
        if log_scale: # if we want to logarithm a feature.
            if any(func_data[feature] == 0) and all(func_data[feature] >= 0): # if we have 0 values and all vals > 0
                x = np.log(func_data[feature] + 1)
            elif any(func_data[feature] == 0) and not all(func_data[feature] >= 0): # if we have 0 and negative vals
                x = np.log(abs(func_data[feature]) + 1)
            else:
                x = np.log(func_data[feature])
        else:
            x = func_data[feature]
    else:
        x = func_data # if data == array or series
        
    quantile_25, quantile_75 = x.quantile(0.25), x.quantile(0.75)
    iqr = quantile_75 - quantile_25
    
    lower_bound = quantile_25 - iqr * left
    upper_bound = quantile_75 + iqr * right
    
    outliers = func_data[(x < lower_bound) | (x > upper_bound)]
    cleaned = func_data[(x >= lower_bound) & (x <= upper_bound)]
    
    return outliers, cleaned.reset_index(drop=True) # .copy() is needed here if we wanna use cleaned daataframe. Or system will swear.

[to contents](#contents)

<h3 id='histboxqq'>Функция рисования 3-х графиков: гистограмма, коробка и Q-Q.</h3>

In [ ]:
def hist_box_qq(arg, full_iqr=False):
    """Function of drawing histogram, box and cuckoo in one bottle.

    full_iqr... It's shorter.. I wrote a function for the lognormal distribution, i.e. there are no outliers on the left.
    But then I thought about it and decided to add it, because then I still log the feature.
    And in general, if the array is usually distributed.
    In a word: full_iqr is Boolean. By default, False. I.e., only the right borders are drawn.
    
    The input is a Series or array.

    Args:
        arg (Series/np.array): any numeric series or array.
        full_iqr (bool, optional): choose, u wanna show only right outliers borders or both. Defaults to False.

    Returns:
        plt
    """
    
    import pandas as pd
    import matplotlib.pyplot as plt
    import statsmodels.api as sm # рисовалка qq-plot.
    import seaborn as sns
    
    func_arg = arg.copy()
    # replace inf.
    func_arg = func_arg.replace([np.inf, -np.inf], np.nan)
    # If there is a nan, I drop it. Because it does not draw if there is a nan.
    func_arg = pd.Series(func_arg).dropna()
    
    fig, ax = plt.subplots(ncols=3, nrows=1, figsize=[18, 5])
    
    # The histogram. And the settings for the histogram lines. Legend = label in each line.
    # Red is the median, green is the tukey, and blue is 3 sigma.
    sns.histplot(func_arg, kde=True, bins=50, ax=ax[0])
    
    ax[0].get_lines()[0].set_color('black')
    ax[0].axvline(func_arg.median(), color='red', linestyle='--', linewidth='1.8', label='median')
    ax[0].axvline(func_arg.quantile(0.75) + ((func_arg.quantile(0.75) - func_arg.quantile(0.25)) * 1.5), color='g', ls='--', lw=2, label='1.5 IQR Tukie')
    ax[0].axvline(func_arg.mean() + 3 * func_arg.std(), color='b', ls='--', lw=2, label='3 IQR z-score')
    ax[0].legend()
    ax[0].set_xlabel(f'Feauture {func_arg.name}')
    
    # Box. And the settings for it.
    sns.boxplot(func_arg, ax=ax[1], orient='h', medianprops={'color': 'red', 'linestyle': '--'})

    ax[1].axvline(func_arg.quantile(0.75) + ((func_arg.quantile(0.75) - func_arg.quantile(0.25)) * 1.5), color='g', ls='--', lw=2, label='1.5 IQR Tukie')
    ax[1].axvline(func_arg.mean() + 3 * func_arg.std(), color='b', ls='--', lw=2, label='3 IQR z-score')
    ax[1].legend()    
    ax[1].set_xlabel(f'Feauture {func_arg.name}')
    
    if full_iqr: # Left emission catch lines are added.
        ax[0].axvline(func_arg.quantile(0.25) - ((func_arg.quantile(0.75) - func_arg.quantile(0.25)) * 1.5), color='g', ls='--', lw=2)
        ax[0].axvline(func_arg.mean() - 3 * func_arg.std(), color='b', ls='--', lw=2)
        ax[1].axvline(func_arg.quantile(0.25) - ((func_arg.quantile(0.75) - func_arg.quantile(0.25)) * 1.5), color='g', ls='--', lw=2)
        ax[1].axvline(func_arg.mean() - 3 * func_arg.std(), color='b', ls='--', lw=2)
    
    # This line builds a Q-Q graph.
    qq = sm.ProbPlot(func_arg, fit=True).qqplot(marker='*', markerfacecolor='b', markeredgecolor='b', alpha=0.3, ax=ax[2])
    # Line, also with the ability to influence color. fmt parameter.
    sm.qqline(qq.axes[2], line='45', fmt='r', linestyle='--')
    
    # The general title.
    plt.suptitle(f'Distribution of the feature: {func_arg.name}').set_fontsize(20)
    
    return plt

[to contents](#contents)

<h3 id='beeswarmhist'>Beeswarm + hist для двух переменных.</h3>

In [ ]:
def swarm_hist(df, targ_col, depend_col, figsize=(15, 5)):
    
    # сырая, есть что добавить в гиперпараметры
    # функция, грубо говоря, про распределение таргета в зависимости от... каких-либо категорий в depend_col
    
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    fig, ax = plt.subplots(ncols=2, figsize=figsize)
    
    # Построение графиков beeswarm + hist
    for i, j in enumerate(df[depend_col].unique()[::-1]):
        y = df[df[depend_col] == j]
        x = np.random.normal(i + 1, 0.1, size=len(y)) # 0.1 отвечает за расстояние между
        ax[0].plot(x, y[targ_col], '*', label=f'{depend_col} {j}', alpha=0.6)
        
        # +i * 12 - чтобы башенки гистограмм не накладывались друг на друга (12 - отвечает за расстояние между)
        sns.histplot(y[targ_col] + i * 12, ax=ax[1], kde=False, 
                     label=f'{depend_col} {j}', binwidth=0.2, color=f"C{i}", alpha=0.5)
    
    ax[0].set_xticks(df[depend_col].unique()[::-1])
    ax[0].set_xlabel(f'{depend_col.capitalize()}')
    ax[0].set_ylabel(f'{targ_col.capitalize()}')
    ax[0].set_title(f'Beeswarm plot of {targ_col.capitalize()} by {depend_col.capitalize()}')
    ax[0].legend()
    
    ax[1].set_title(f'Histogram of {targ_col.capitalize()} by {depend_col.capitalize()}')
    ax[1].set_xticks([])
    ax[1].set_xlabel(f'{targ_col.capitalize()}')
    ax[1].set_ylabel('Count')
    ax[1].legend()
    
    return plt

[to contents](#contents)

<h3 id='mycorr'>Функция рисования тепловой карты корреляции.</h3>

In [ ]:
def my_corr(df, targ_col=False, method='pearson', corrneg=0, corrpos=0, figsize=(10, 5), xtickrot=90, annot_kws={'size': 'medium'}, show_high_corr_cols_names=False, ax=None):
    """
    The function of drawing a correlation graph, but not an ordinary one, but one in which the lower part is the entire one,
    and the upper part contains only those correlation values that are greater and less than some number we have specified.

    Args:
        df (pd.Dataframe): any df.
        targ_col (str, optional): if targ_col is set, func just move this col into the end of dataframe, to better visualisation. It will be at the last row of the plot. Defaults: False
        method (str, optional): method u like. Default: pearson.
        corrneg (float, optional): negative threshold for display. (for negative correlation value)
        corrpos (float, optional): positive threshold. (for a positive value) both default to 0. At 0, the full standard correlation matrix will be displayed.
        figsize (tuple, optional): u know what it is.)
        xtickrot(int, optional): rotate x axis labels. Default: 90
        annot_kws(dict, optional): size of annot font. Default: {"size": 'medium'}
        show_high_corr_cols_names(bool, optional): if we wanna watch names of high corr cols. Default: False. Should use without plot and other plot args. Just thresholds and this one.
        ax (ax, optional): use only if u wanna add this plot to your multiple plots. Default: None

    Returns:
        _type_: plt or df
    """
    
    import pandas as pd
    import numpy as np
    import matplotlib
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    if targ_col:
        # sort indexes by correlation with targ_col from highest to lowest
        most_corr_cols_index = df.corr(numeric_only=True, method=method)[targ_col].drop(targ_col).sort_values(ascending=False).index
        # putting targ_col to fist row (y = 1) of the chart with sorted features (watch ur chart and u will get it)
        corr_df = pd.concat([df.drop(targ_col, axis=1)[most_corr_cols_index], df[targ_col]], axis=1).corr(numeric_only=True, method=method).copy()
    else:
        corr_df = df.corr(numeric_only=True, method=method).copy()
    
    # We find indices where the correlation value is greater and less than a certain number.
    high_corr_pairs = ((corr_df >= corrpos) & (corr_df <= 1)) | (corr_df <= corrneg)
    
    # We receive only those pairs of features where the condition is met. The rest are NaN.
    high_corr_features = corr_df[high_corr_pairs]
    
    # if we wanna watch high corr pairs in the form of a df. Without picture. I decided 2 split it up.
    if show_high_corr_cols_names:
        # stack hight corr feature.
        shccn = high_corr_features.stack().to_frame().reset_index().rename(columns={0: 'corr'})
        # sort'em.
        shccn[['level_0', 'level_1']] = np.sort(shccn[['level_0', 'level_1']], axis=1)
        # keep only 1 pair of high corr cols names.
        shccn = shccn.groupby(['level_0', 'level_1']).first().reset_index()
        # removing the diagonal.
        shccn = shccn.query('level_0 != level_1')
        # return with resetting index.
        return shccn.sort_values(by='corr', ascending=False).reset_index(drop=True)
    
    # if we wanna see the pic.
    else:
        if ax is None:
            fig, ax = plt.subplots(figsize=figsize)
    
        ax.set_title(f'Correlation of incoming dataframe features.', pad=10) # узкое место pad=10, чтоб заголовок был на расстоянии от графика
        
        # 2 masks, one triu (tri upper), the second tril (tri lower).
        mask_for_logging_df = np.triu(np.ones_like(corr_df, dtype=bool))
        mask_for_logging_df_2 = np.tril(np.ones_like(high_corr_features, dtype=bool))
        
        # 2 graphics. I didn’t figure out how to make such a picture using 1 chart. That's why cbar=False helps.
        sns.heatmap(corr_df, annot=True, annot_kws=annot_kws, cmap='coolwarm', mask=mask_for_logging_df, vmax=1, vmin=-1, linewidths=0.1, fmt='.2f', ax=ax) # <--- lower part
        
        # if condition == 1 high_corr_features contains only nan, so we dont need upper part. 
        # or.. high_corr_features has no nan, i.e. corrpos and corrneg == 0, so we draw all corr matrix.
        if (len(high_corr_features.isna().sum().value_counts()) != 1) | ((corrpos == 0) & (corrneg == 0)):
            sns.heatmap(high_corr_features, annot=True, annot_kws=annot_kws, cmap='coolwarm', mask=mask_for_logging_df_2, cbar=False, vmax=1, vmin=-1, linewidths=0.1, fmt='.2f', ax=ax) # <--- upper part
        
        ax.set_xticklabels(ax.get_xticklabels(), rotation=xtickrot)
    
        ax.plot([0, len(df.columns)], [0, len(df.columns)], color='black', linewidth=2); # It's just a line. I don't know why. But i like it.
        
        return plt

[to contents](#contents)

<h3 id='fillnadict'>Функция, создающая словарь для заполнения метода .fillna</h3>

In [ ]:
def fillna_dict_for_custom_df(df, func, targ_col=None, roundnum=10):
    """Function for generating a dictionary for the fillna function argument.
    
    The function takes as input a dataframe with np.nan and the function that we want to aggregate.
    That is, we have a feature in which there is a nan. We want to fill the nan, for example, with the median.
    We run through the names of the columns that have nan and assign each nan a median value for the column.
    Why the condition? When we ask for mode, we get a Series object that needs to be accessed by index to get the value.
    And when we ask for the average or median, it immediately returns the value.

    Args:
        df (pd.Dataframe): any df
        func (str): any func in str format. 
        targ_col (str, optional): name of target column by which we wanna groupby and have needed agregation. Defaults: None.
        roundnum (int, optional): num of np.round(X, ?), how we want to round our outcoming number. Defaults: 10.
        
        Ex: df.fillna(value=fillna_dict_for_custom_df(df, 'mean'))

    Returns:
        _type_: dict
    """
    
    if targ_col:
        return {i: df.groupby(targ_col)[i].transform(lambda x: x.mode().iloc[0]) if np.issubdtype(df[i], 'object') or func == 'mode' else np.round(df.groupby(targ_col)[i].transform(func), roundnum) for i in df.loc[:, df.isna().mean() > 0].columns}
    return {i: df[i].mode().iloc[0] if np.issubdtype(df[i], 'object') or func == 'mode' else np.round(df[i].agg(func), roundnum) for i in df.loc[:, df.isna().mean() > 0].columns}

[to contents](#contents)

<h3 id='decoratexlabels'>Функция декорирования оси Х столбчатой диаграммы.</h3>

In [ ]:
## ОРИГИНАЛ!
#from itertools import groupby # для рисования мультииндекс меток по оси х (и не только, но здесь - только).
#
## Чумовейшие три функции, которые рисуют мультииндекс разметку по оси Х!!!
## Работа происходит по третьей функции. 
## На вход подаётся ax и датафрейм вида: 
## 1. главный индекс; 
## 2. второстепенный индекс или группа индексов, по иерархии от старшего к младшему; 
## 3. последним идёт индекс, который является оттенком (hue).
## Вид группировки: .groupby(['1_st_order_idx', '2_nd_order_idx', ..., 'hue_idx'])['nums_column'].mean().unstack()
## P.S. для работы нужен groupby из itertools
#
#def add_line(ax, xpos, ypos):
#    line = plt.Line2D([xpos, xpos], [ypos + .1, ypos],
#                      transform=ax.transAxes, color='gray')
#    line.set_clip_on(False)
#    ax.add_line(line)
#
#def label_len(my_index,level):
#    labels = my_index.get_level_values(level)
#    return [(k, sum(np.fromiter((1 for i in g), dtype=int))) for k, g in groupby(labels)]
#
#def label_group_bar_table(ax, df):
#    ypos = -.1
#    scale = 1./df.index.size
#    for level in range(df.index.nlevels)[::-1]:
#        pos = 0
#        for label, rpos in label_len(df.index, level):
#            lxpos = (pos + .5 * rpos)*scale
#            ax.text(lxpos, ypos, label, ha='center', transform=ax.transAxes)
#            add_line(ax, pos*scale, ypos)
#            pos += rpos
#        add_line(ax, pos*scale ,ypos)
#        ypos -= .1
        
def label_group_bar_table(ax, df, orient='v'):
    """label_group_bar_table 

    Чумовейшие три функции, которые рисуют мультииндекс разметку по оси Х!!!
    На вход подаётся ax и датафрейм вида: 
     1. главный индекс; 
     2. второстепенный индекс или группа индексов, по иерархии от старшего к младшему; 
     3. последним идёт индекс, который является оттенком (hue).
     Вид группировки: .groupby(['1_st_order_idx', '2_nd_order_idx', ..., 'hue_idx'])['nums_column'].mean().unstack()
     P.S. для работы нужен groupby из itertools
     P.S.2 Сие ваяние не моё, потому хз, что там происходит и я не распишу каждую строчку, но результат меня устраивает.)

    Args:
        ax (_type_): ax with ur plot.
        df (pd.DataFrame): ur df which u wanna plot.
        orient (str, optional): if we wanna draw horisontal plot, just set not 'v' parameter. Defaults: 'v'.
    """
    from itertools import groupby # для рисования мультииндекс меток по оси х (и не только, но здесь - только).
    import matplotlib.pyplot as plt
    
    def add_line(ax, xpos, ypos):
        line = plt.Line2D([xpos, xpos], [ypos + .1, ypos],
                          transform=ax.transAxes, color='gray')
        line.set_clip_on(False)
        ax.add_line(line)
        
    def add_line_h(ax, xpos, ypos):
        line = plt.Line2D([xpos + .1, xpos], [ypos, ypos],
                          transform=ax.transAxes, color='gray')
        line.set_clip_on(False)
        ax.add_line(line)
    
    def label_len(my_index, level):
        labels = my_index.get_level_values(level)
        return [(k, sum(np.fromiter((1 for i in g), dtype=int))) for k, g in groupby(labels)]
    
    
    if orient == 'v':    
        ypos = -.1
        scale = 1./df.index.size
        for level in range(df.index.nlevels)[::-1]:
            pos = 0
            for label, rpos in label_len(df.index, level):
                lxpos = (pos + .5 * rpos)*scale
                ax.text(lxpos, ypos, label, ha='center', transform=ax.transAxes)
                add_line(ax, pos*scale, ypos)
                pos += rpos
            add_line(ax, pos*scale, ypos)
            ypos -= .1
    else:
        xpos = -.1
        scale = 1./df.index.size
        for level in range(df.index.nlevels)[::-1]:
            pos = 0
            for label, rpos in label_len(df.index, level):
                lypos = (pos + .3 * rpos)*scale
                ax.text(xpos, lypos, label, ha='left', transform=ax.transAxes)
                add_line_h(ax, xpos, pos*scale)
                pos += rpos
            add_line_h(ax, xpos, pos*scale)
            xpos -= .1

####################################################################################################################
# Пример использования:

fig, ax = plt.subplots(figsize=(30, 8))

plt.suptitle('Динамика соотношения среднего годового дохода в зависимости от формата занятости, года и опыта').set_fontsize(25)

# Подготовка датафрейма.
df_for_this_cell = df_for_this_cell.groupby([choose_job, 'remote_ratio', 'work_year', 'experience_level'])['salary_in_usd'].mean().unstack()

# Сама строка.
df_for_this_cell.plot(kind='bar', ax=ax, stacked=True, legend=True)

ax.set_xticklabels('') # обязательно удаление названий по оси х
ax.set_xlabel('')

# И оборачивание подготовленного, сгруппированного датафрейма в функцию
# На самом деле можно и без этого. Функция чисто декоративная.
label_group_bar_table(ax, df_for_this_cell)
####################################################################################################################

[to contents](#contents)

<h3 id='recprecfthreshplot'>Функция рисования порогов вероятности для метрик recall, precision и f1_score</h3>

In [ ]:
def rpf_threshold_dependence(y, y_proba, thresholds=np.arange(0.1, 1, 0.05), figsize=(10, 4), xtickrot=0):
    """rpf_threshold_dependence

    Рисовалка графиков зависимости метрик от порога вероятности.

    Args:
        y (pd.Series): правильные ответы.
        y_proba (pd.Series): предсказанные ответы.
        thresholds (numpy.ndarray, optional): пороги вероятностей. Defaults to np.arange(0.1, 1, 0.05).
        figsize (tuple): figsize. Defaults to (10, 4).
        xtickrot (int): rotation of x labels. Defaults to 0.

    Returns:
        plt: rpf_threshold_dependence.show()
    """
    from sklearn import metrics
    import matplotlib.pyplot as plt
    
    # Создадим списки, в которых будем хранить значения метрик.
    recall_scores = []
    precision_scores = []
    f1_scores = []
    
    # В цикле перебираем значения порогов вероятности
    for threshold in thresholds:
        # Делаем предсказание
        y_pred = y_proba.apply(lambda x: 1 if x > threshold else 0)
        
        #Считаем метрики и добавляем их в списки
        recall_scores.append(metrics.recall_score(y, y_pred))
        precision_scores.append(metrics.precision_score(y, y_pred, zero_division=0)) # Здесь поставил zero_div=0, т.к. иногда вылезает предупреждение.
        f1_scores.append(metrics.f1_score(y, y_pred))
    
    # Визуализируем метрики при различных threshold.
    
    fig, ax = plt.subplots(figsize=figsize)
    
    # Строим линейный график зависимости recall от threshold.
    ax.plot(thresholds, recall_scores, label='Recall')
    # Строим линейный график зависимости precision от threshold.
    ax.plot(thresholds, precision_scores, label='Precision')
    # Строим линейный график зависимости F1 от threshold.
    ax.plot(thresholds, f1_scores, label='F1-score')
     
    # Даём графику название и подписываем оси.
    ax.set_title('Recall/Precision/F1 dependence on the threshold')
    ax.set_xlabel('Probability threshold')
    ax.set_ylabel('Score')
    # Устанавливаем отметки по оси x.
    ax.set_xticks(thresholds)
    # Добавил поворот меток оси х. Иногда полезно.
    ax.set_xticklabels(ax.get_xticklabels(), rotation=xtickrot) 
    # Отображаем легенду.
    ax.legend();
    
    return plt

[to contents](#contents)

<h3 id='kmeansinertiasilhouette'>Функция рисования нахождения оптимального К для алгоритма K-MEANS с помощью inertia и silhouette</h3>

In [ ]:
def inertia_silhouette_plot(X, alg, max_range=10, coef='silhouette', figsize=(12, 5)):
    """inertia_silhouette_plot 

    Функция рисования графиков инерции и силуэта для определения опимального количества кластеров для алгоритма k-means.
    
    Args:
        X (pd.DataFrame): incoming X. (X should be DataFrame object)
        alg (sklearn.cluster._kmeans.KMeans): incoming alg.
        max_range (int, optional): max number of iterations. Defaults to 10.
        coef (str): choose what we wanna see: inertia graph, silhouette graph, ot both. Defaults to 'silhouette'.
        figsize (tuple, optional): figsize of picture. Defaults to (12, 5).

    Returns:
        plt: plt.show()
    """
    # import nessessary lib for copy incoming alg.
    import copy
    from sklearn.metrics import silhouette_score
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    if coef == 'inertia':
        # set figax.
        fig, ax = plt.subplots(1, 1, figsize=figsize)
        # copy of incoming alg. It needs here.
        func_alg = copy.deepcopy(alg)
        # Series for plot.
        inertia_series = pd.Series([func_alg.set_params(n_clusters=i).fit(X).inertia_ for i in range(1, max_range)], index=range(1, max_range))
        # plot.
        sns.lineplot(inertia_series, ax=ax, marker='X')
        # plot settings.
        plt.suptitle('Inertia of incoming X')
        ax.set_xlabel("cluster", fontsize=12), ax.set_ylabel("inertia", fontsize=12)
        ax.set_xticks(range(1, max_range, 1)), ax.set_yticks(inertia_series)
        
        return plt
        
    elif coef == 'silhouette':
        # set figax.
        fig, ax = plt.subplots(1, 1, figsize=figsize)
        # copy of incoming alg. It needs here.
        func_alg = copy.deepcopy(alg)
        # Series for plot.
        silhouette_series = pd.Series([silhouette_score(X, func_alg.set_params(n_clusters=i).fit(X).labels_) for i in range(2, max_range)], index=range(2, max_range))
        # plot.
        sns.lineplot(silhouette_series, ax=ax, marker='o')
        # plot settings.
        plt.suptitle('Silhouette of incoming X')
        ax.set_xlabel("cluster", fontsize=12), ax.set_ylabel("silhouette", fontsize=12)
        ax.set_xticks(range(1, max_range, 1)), ax.set_yticks(silhouette_series)
        
        return plt
    
    else:
        # set figax.
        fig, ax = plt.subplots(1, 2, figsize=figsize)
        # copy of incoming alg. It needs here.
        func_alg = copy.deepcopy(alg)
        # 2 series. 1 = inertia series, 2 = silhouette series.
        inertia_series = pd.Series([func_alg.set_params(n_clusters=i).fit(X).inertia_ for i in range(1, max_range)], index=range(1, max_range))
        silhouette_series = pd.Series([silhouette_score(X, func_alg.set_params(n_clusters=i).fit(X).labels_) for i in range(2, max_range)], index=range(2, max_range))
        # plotting.
        sns.lineplot(inertia_series, ax=ax[0], marker='X')
        sns.lineplot(silhouette_series, ax=ax[1], marker='o')
        
        plt.suptitle('Inertia and silhouette of incoming X')
        
        fig.tight_layout(w_pad=5, h_pad=3) # расстояние между подграфиками
        plt.subplots_adjust(top=0.9) # чтобы заголовок не налезал на графики
        
        # set x and y labels.
        ax[0].set_xlabel("cluster", fontsize=12), ax[0].set_ylabel("inertia", fontsize=12)
        ax[1].set_xlabel("cluster", fontsize=12), ax[1].set_ylabel("silhouette", fontsize=12)
        
        # set x and y ticks bins.
        ax[0].set_xticks(range(1, max_range, 1)), ax[0].set_yticks(inertia_series)
        ax[1].set_xticks(range(1, max_range, 1)), ax[1].set_yticks(silhouette_series)
        
        del func_alg, inertia_series, silhouette_series
        
        return plt

[to contents](#contents)

<h3 id='educurvebuildfunc'>Функция построения кривой обучения</h3>

In [ ]:
def plot_learning_curve(model, X, y, cv, scoring='f1', ax=None, title=''):
    
    from sklearn import model_selection #методы разделения и валидации
    import numpy as np
    import matplotlib.pyplot as plt
    
    # Вычисляем координаты для построения кривой обучения
    train_sizes, train_scores, valid_scores = model_selection.learning_curve(estimator = model, # модель
                                                                             X = X, # матрица наблюдений X
                                                                             y = y, # вектор ответов y
                                                                             cv = cv, # кросс-валидатор
                                                                             scoring = scoring) # метрика
    # Вычисляем среднее значение по фолдам для каждого набора данных
    train_scores = np.mean(train_scores, axis=1)
    valid_scores = np.mean(valid_scores, axis=1)
    
    # Если координатной плоскости не было передано, создаём новую
    if not ax:
        fig, ax = plt.subplots(figsize=(10, 4))
    
    ax.plot(train_sizes, train_scores, label='Train') # Строим кривую обучения по метрикам на тренировочных фолдах
    ax.plot(train_sizes, valid_scores, label='Valid') # Строим кривую обучения по метрикам на валидационных фолдах
    
    ax.set_title(f'Learning curve: {title}')
    ax.set_xlabel('Train data size')
    ax.set_ylabel('Score')
    ax.xaxis.set_ticks(train_sizes) # Устанавливаем отметки по оси абсцисс
    ax.set_ylim(0, 1) # Устанавливаем диапазон оси ординат
    ax.legend()

[to contents](#contents)

<h3 id='prcurve'>Функция построения precision-recall кривой с меткой наилучшего порога (PR-кривая)</h3>

In [ ]:
def pr_curve(y, y_proba_pred, ax=None, title='', toggle_pic_or_threshnum='pic'):
    """pr_curve 
    
    Графическая функция для отображения кривой precision-recall с возможностью выводить либо график, либо наилучший порог.

    Args:
        y (pd.Series): y_train or any u like.
        y_proba_pred (numpy.ndarray): proba_pred.
        ax (?, optional): this is just ax. Defaults to None.
        title (str, optional): title. Defaults to ''.
        toggle_pic_or_threshnum (str, optional): _description_. Defaults to 'pic'.

    Returns:
        plt or float: u will recieve either a pic or a best threshold num.
    """
    
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn import metrics
    
    # Мне нравится пандас, потому сделаю через датафрейм
    prc = pd.DataFrame(metrics.precision_recall_curve(y, y_proba_pred), index=['precision', 'recall', 'thresholds']).T
    
    # Вычисляем F1-меру при различных threshold.
    f1_scores = (2 * prc['precision'] * prc['recall']) / (prc['precision'] + prc['recall'])
    # Определяем индекс максимума.
    idx = np.argmax(f1_scores)
    
    # Условие для: либо рисуем график... 
    if toggle_pic_or_threshnum == 'pic':
        # если не указан ax
        if not ax:
            fig, ax = plt.subplots(figsize=(10, 5))
        # Линия и точка на линии
        ax.plot(prc['recall'], prc['precision'], label=f"{title} auc: {metrics.auc(prc['recall'], prc['precision']):.2f}")
        ax.scatter(prc.loc[idx, 'recall'], prc.loc[idx, 'precision'], c='black', marker='o', s=80, label='Best F1 score')
        
        # Добавляем аннотацию с меткой точки на оси X (порог и ф1_скор)
        recall_best = prc.loc[idx, 'recall'] # координаты точки
        precision_best = prc.loc[idx, 'precision']
        ax.annotate(f"Threshold: {prc.loc[idx, 'thresholds']:.2f}\nF1_score: {f1_scores[idx]:.2f}", 
                    xy=(recall_best+0.01, precision_best), 
                    xytext=(recall_best + 0.05, precision_best),  # Смещаем текст немного вправо
                    arrowprops=dict(facecolor='black', shrink=0.05, width=0.7, headwidth=7, headlength=7))
        # Метки и заголовок.
        ax.set_xlabel('Recall')
        ax.set_ylabel('Precision')
        ax.set_title(f'Precision-recall curve: {title}')
        
        plt.legend();
    # либо выводим наилучший порог метрики.
    else:
        return prc.loc[idx, 'thresholds']

[to contents](#contents)

<h3 id='shapirowilkfunc'>Функция проверки на нормальность Шапиро-Уилка.</h3>

In [ ]:
import pandas as pd
import scipy.stats as stats

alpha = 0.5

# Функция проверки на нормальность Шапиро-Уилка.
def shapirowilk(df_or_arr, alpha=0.05):
    """Продвинутая функция проверки нормальности Шапиро-Уилка.
    
    Тут всё просто. Если датафрейм, то формируем через цикл списки названий нормальных и не нормальных столбцов.
    Иначе сразу печатаем реультат теста. Т.к. у нас Series или массив.
    Далее смотрим на длины списков и формируем список выводов функции. И выводим его.
    Почему продвинутая? Потому что я раньше считал количество норм и не норм столбцов через: dict(Counter([shapirowilk(df_for_this_cell[i]) for i in df_for_this_cell.columns]))
    а сама функция выглядела, как функция Левена ниже.))

    Args:
        df_or_arr (pd.DataFrame): Dataframe or Series or array
        alpha (float): alpha significance level

    Returns:
        df or str
    """
    func_df = pd.DataFrame()

    if isinstance(df_or_arr, pd.DataFrame):
        for i, j in enumerate(df_or_arr.columns):
            if stats.shapiro(df_or_arr[j])[1] <= alpha:
                func_df.loc[i, 'Distribution'] = df_or_arr[j].name
                func_df.loc[i, 'Status'] = 'не нормальное'
            else:
                func_df.loc[i, 'Distribution'] = df_or_arr[j].name
                func_df.loc[i, 'Status'] = 'нормальное'
    else:
        return f'Распределение не нормальное. При alpha = {alpha}' if stats.shapiro(df_or_arr)[1] <= alpha else f'Распределение нормальное. При alpha = {alpha}'
    
    return func_df

[to contents](#contents)

<h3 id='levenefunc'>Функция проверки равнества дисперсий от Левена.</h3>
(недоработанная)

In [ ]:
# Функция проверки на равенство дисперсий от Левена.
def levene(*args):
    return 'Дисперсии не равны. Нужно использовать непараметрический тест. При alpha = 0.05' if stats.levene(*args)[1] <= alpha else 'Дисперсии равны. При alpha = 0.05'

[to contents](#contents)

<h3 id='maintagsworkfunc'>Функция для работы с ["строками", "внутри", "списка тэгов", "1 Adult", "2 Adults"]</h3>
ОСНОВНАЯ.

In [ ]:
def transform_specific_string(tags_column, words_to_transform, list_of_exceptions=[], hard_split_list=[], replacement_dict=False):
    from tqdm.notebook import tqdm_notebook
    """
    Функция преобразования str элементов списка.
    Типа когда у нас есть список, в котором есть строки, которые нужно побить по какому-то слову или словосочетанию.
    И вернуть обратно такой же список, но с побитыми строками. 
    P.S. Почему не сделать изначально весь столбец .lower() и потом использовать функцию?
    Приятней же искать в выводе файла 'output_transformed.txt' следующие слова на обрезку, которые начинаются с заглавной буквы.
    Ну, лично мне. Можно изначально сделать всё lower и оно всё равно будет работать.

    Args:
        tags_column (series): столбец со списками, которые с элементами str.
        split_word (str): ключевое слово или словосочетание.
        list_of_exceptions (list): список исключений, которые мы не хотим видеть в итоговом списке.
        hard_split_list (list): список слов, по которым строка будет делиться, не склеивая остатки (with, and, etc).
        replacement_dict (dict): словарь замены одинакового, написанного по-разному.

    Returns:
        list: возвращаем побитый список.))
        
    Пример использования: 
    нужно вставить код ниже в ячейку после функции и поменять your_df[tags_col] на свой дф с столбцом тэгов и везде заменить your_df (выделить your_df и на клаве: ctrl+D или cmd+D если мак). 
    ----------------------------------------------------------------------------------------------------------
        words_to_transform = [# любые слова или словосочетания, по которым хотим побить каждую строку, порядок важен] # типа сначала сложносоставные слова "1 2 Adult", а потом "2 Adult". У себя я разбил на 2 тего: "1 2 Adult" - 1 адулт и 2 адулт. Чувак же указал.
        list_of_exceptions = [# слова исключения, которым в списках не место (with, and)] # Тут порядок не важен.
        hard_split_list = [# слова, по которым строка будет делиться, без склеивания остатков (with, and, etc)]
        replacement_dict = {# словарь, если хотим заменить какие-то слова или словосочетания} # И тут тоже.
        
        # Помещаем преобразование в отдельный столбец.
        your_df['transformed'] = transform_specific_string(your_df[tags_col], words_to_transform=words_to_transform, list_of_exceptions=list_of_exceptions, replacement_dict=replacement_dict)
        
        # vvvvvvvvvvvvvvvvvvv  Ниже работа с выводом  vvvvvvvvvvvvvvvvvvv
        # Если хотим посмотреть, что у нас получилось используем следующее:
        # Полезная штука, если данных много и в аутпуте всё не помещается.
        # В текущей директории создаётся файл в котором находится список уникальных элементов столбца. Ну или не совсем уникальных, если не делали .lower() Нужно вставить после .explode()
        output_data = list(set(your_df['transformed'].explode()))
        
        with open('output_transformed.txt', 'w') as f:
            for item in output_data:
                f.write("%s\n" % item)
        
        # Можно проверить, как оно бьётся по конкретному слову, словосочетанию:
        print(your_df.loc[[' 2 adult '.lower() in str(i) for i in your_df['tags'].apply(lambda x: [i.lower() for i in x])]])
        
        # Можно вывести 2 столбца со списками и посмотреть, как идут дела.)
        your_df.loc[:, ['tags', 'transformed']]
        
        # Если есть ещё одна функция, которая делает то же самое, то можно проверить столбцы на не равенство после работы двух функций. 
        # Всё должно быть False. И если есть True, убираем вальюкаунтс и спереди добавляем df.loc[]. И смотрим, что где плохо бьётся.
        (df_main['transformed'].apply(lambda x: [i.lower() for i in x]) != df_main['transformed_2'].apply(lambda x: [i.lower() for i in x])).value_counts()
    ----------------------------------------------------------------------------------------------------------
    """
    # Внутренняя функция, которая лупит конкретную строку.
    def transform(row, split_word, list_of_exceptions, hard_split_list, replacement_dict):
        # Возвращаемый список.
        new_row = []
        # Цикл по элементам строки.
        for item in row:
            # Создаём временную переменную, которая является элементом списка + пробелы с двух сторон, потому что реплейс мы делаем по отдельно стоящему слову, т.е. слову, которое окружено с двух сторон пробелами.
            temp_item = ' ' + item.lower() + ' '
            # Условие на вхождение слова разделителя в элемент списка. Я сначала сделал через re.search, но re работает дольше, потому переписал через вот так.
            if split_word.lower() in item.lower():
                # Если это слова, по которым нужно именно разделить на отдельные части, то:
                if split_word.lower() in hard_split_list:
                    # Заменяем слово разделитель на |||||
                    temp_item = temp_item.replace(f' {split_word.lower()} ', '|||||')
                    # Если реплейс сработал то:
                    if '|||||' in temp_item:
                        # Делим по реплесу.
                        split_items = temp_item.split('|||||')
                        # Добавляем само слово разделитель и остатки после разделения - отдельно.
                        new_row.extend([split_word.strip(), *[i.strip() for i in split_items]])
                    else:
                        # Иначе записываем просто итем.
                        new_row.append(item.strip())
                # Если же это другие слова или словосочетания, то:
                else:
                    # Заменяем разделитель на |||||.
                    temp_item = temp_item.replace(f' {split_word.lower()} ', '|||||')
                    # Тут как раз проверка на "сработал реплейс или нет". 
                    # У меня был момент: входящая строка: 'Two Connecting Superior Double Rooms', слово-разделитель: 'Superior Double Room'. 
                    # Входит? Входит. Реплейсит? - нет. Ничего не возвращает и я добавляю просто слово разделитель, упуская саму строку. 
                    # Потому сначала вот это условие.
                    if '|||||' in temp_item:
                        # Если да, то бьём и
                        split_items = temp_item.split('|||||')
                        # добавляем само слово разделитель, и склеиваем остатки по пробелу.
                        new_row.extend([split_word.strip(), ' '.join([i.strip() for i in split_items])])
                    else:
                        # Иначе записываем просто итем.
                        new_row.append(item.strip())
            else:
                # Как и здесь.
                new_row.append(item.strip())
                
        # Чистим список.
        new_row = [item for item in new_row if item != '' and item not in list_of_exceptions]
        # Переименовываем элементы списка только тогда, когда это нужно и переименование зависит от слова-разделителя.
        if replacement_dict and split_word in replacement_dict:
            new_row = [replacement_dict[split_word] if split_word in item else item for item in new_row]

        return new_row
    
    ##########vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv##############
    # Общий прогресс прогресс бара от 0 до 100% и Инициализация прогресс бара с настройками отображения.
    total_progress = 100
    with tqdm_notebook(total=total_progress, desc=f"Cell progress: ", bar_format='{desc}: {percentage:.0f}%|{bar}| {n:.0f}/{total_fmt} [Elapsed: {elapsed} m/s, Remaining: {remaining} m/s]') as pbar:
        cumulative_progress = 0 # Чтобы прогресс бар работал, как надо и завершался всегда на 100.
    ##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############
        
        # Здесь сам цикл, в котором мы перебираем каждое слово и передаём его в каждую строку, которая является списком.))
        # И обрабатываем это слово с помощью функции transform для каждой строки.
        for i, word in enumerate(words_to_transform):
            tags_column = tags_column.apply(transform, split_word=word, list_of_exceptions=list_of_exceptions, hard_split_list=hard_split_list, replacement_dict=replacement_dict)
            
    ##########vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv##############        
            # Рассчитываем размер шага на основе оставшегося прогресса и оставшихся итераций.
            remaining_iterations = len(words_to_transform) - i # вот тут пригодился индекс i
            remaining_progress = total_progress - cumulative_progress
            step_size = remaining_progress / remaining_iterations
            
            # Обновляем внутренний и общий прогресс.
            cumulative_progress += step_size
            pbar.update(step_size)
    ##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############

    return tags_column

[to contents](#contents)

<h3 id='retagsworkfunc'>Функция для работы с ["строками", "внутри", "списка тэгов", "1 Adult", "2 Adults"]</h3>
Второстепенная с использованием модуля re.

In [ ]:
def transform_specific_string_2(tags_column, words_to_transform, list_of_exceptions=[], replacement_dict=False): # Рабочий вариант №2
    from tqdm.notebook import tqdm_notebook
    import re
    """
    Функция преобразования str элементов списка.
    Типа когда у нас есть список, в котором есть строки, которые нужно побить по какому-то слову или словосочетанию.
    И вернуть обратно такой же список, но с побитыми строками. 
    P.S. Почему не сделать изначально весь столбец .lower() и потом использовать функцию, но.. 
    Приятней же искать в выводе файла 'output_transformed.txt' следующие слова на обрезку, которые начинаются с заглавной буквы.
    Ну, лично мне. 

    Args:
        tags_column (series): столбец со списками, которые с элементами str.
        split_word (str): ключевое слово или словосочетание.
        list_of_exceptions (list): список исключений, которые мы не хотим добавлять при сплите: with, and.
        replacement_dict (dict): словарь замены одинакового, написанного по-разному.

    Returns:
        list: возвращаем побитый список.))
        
    Пример использования: 
    нужно вставить код ниже в ячейку после функции и поменять your_df[tags_col] на свой дф с столбцом тэгов и везде заменить your_df (выделить your_df и на клаве: ctrl+D или cmd+D если мак). 
    ----------------------------------------------------------------------------------------------------------
        words_to_transform = [# любые слова или словосочетания, по которым хотим побить каждую строку, порядок важен] # типа сначала сложносоставные слова "1 2 Adult", а потом "2 Adult". У себя я разбил на 2 тего: "1 2 Adult" - 1 адулт и 2 адулт. Чувак же указал.
        list_of_exceptions = [# слова исключения, которым в списках не место (with, and)] # Тут порядок не важен.
        replacement_dict = {# словарь, если хотим заменить какие-то слова или словосочетания} # И тут тоже.
        
        # Помещаем преобразование в отдельный столбец.
        your_df['transformed'] = transform_specific_string(your_df[tags_col], words_to_transform=words_to_transform, list_of_exceptions=list_of_exceptions, replacement_dict=replacement_dict)
        
        # vvvvvvvvvvvvvvvvvvv  Ниже работа с выводом  vvvvvvvvvvvvvvvvvvv
        # Если хотим посмотреть, что у нас получилось используем следующее:
        # Полезная штука, если данных много и в аутпуте всё не помещается.
        # В текущей директории создаётся файл в котором находится список уникальных элементов столбца. Ну или не совсем уникальных, если не делали .lower() Нужно вставить после .explode()
        output_data = list(set(your_df['transformed'].explode()))
        
        with open('output_transformed.txt', 'w') as f:
            for item in output_data:
                f.write("%s\n" % item)
        
        # Можно проверить, как оно бьётся по конкретному слову, словосочетанию:
        print(your_df.loc[[' 2 adult '.lower() in str(i) for i in your_df['tags'].apply(lambda x: [i.lower() for i in x])]])
        
        # Можно вывести 2 столбца со списками и посмотреть, как идут дела.)
        your_df.loc[:, ['tags', 'transformed']]
        
        # В эту версию я добавлю отладчик, т.к. основная версия другая, без модуля re.
        new_row = []
        item = 'Two Connecting Superior Double Rooms'
        split_word_list = ['Superior Double Room']
        
        for split_word in split_word_list:
            temp_item = ' ' + item.lower() + ' '
            
            if split_word.lower() in item.lower():
                if split_word.lower() == 'with' or split_word.lower() == 'and':
                    temp_item = temp_item.replace(f' {split_word.lower()} ', '|||||')
                    split_items = temp_item.split('|||||')
                    new_row.extend([split_word.strip(), *[i.strip() for i in split_items]])
                else:
                    
                    temp_item = temp_item.replace(f' {split_word.lower()} ', '|||||')
                    
                    if '|||||' in temp_item:
                        print(temp_item) # Принтами можно проверить каждый шаг.
                        split_items = temp_item.split('|||||')
                        new_row.extend([split_word.strip(), ' '.join([i.strip() for i in split_items if len(i) > 1])])
                    else:
                        new_row.append(item.strip())
            else:
                new_row.append('xyu')
        new_row
        
        # Проверяем на равенство после работы двух функций.
        (df_main['transformed'].apply(lambda x: [i.lower() for i in x]) != df_main['transformed_2'].apply(lambda x: [i.lower() for i in x])).value_counts()
    ----------------------------------------------------------------------------------------------------------
    """
    # Внутренняя функция, которая лупит конкретную строку.
    def transform(row, split_word, list_of_exceptions, replacement_dict):
        # Возвращаемый список.
        new_row = []
        # Цикл по элементам строки.
        for item in row:
            # Используется модуль re для поиска слова, которое окружено с двух сторон пробелами и проверяет его на вхождение в элемент списка.
            if re.search(r'\b' + split_word.lower() + r'\b', item.lower()):
                # Если это слова, по которым нужно именно разделить на отдельные части, то:
                if split_word.lower() == 'with' or split_word.lower() == 'and':
                    # бьём по нему элемент списка и
                    split_items = item.lower().split(split_word.lower())
                    # добавляем само слово разделитель и остальное, что осталось в разбитом элементе.
                    new_row.extend([split_word.strip(), *[i.strip() for i in split_items]])
                # Если же это другие слова или словосочетания, то:
                else:
                    # бьём,
                    split_items = item.lower().split(split_word.lower())
                    # добавляем само слово и склеиваем остатки.
                    new_row.extend([split_word.strip(), ' '.join([i.strip() for i in split_items])])
            # Если же слово не совпадает:
            else:
                # то просто добавляем элемент списка неизменным.
                new_row.append(item.strip())
        
        # Чистим строку.
        new_row = [item for item in new_row if len(item) > 2 and item not in list_of_exceptions]
        # Переименовываем элементы списка только тогда, когда это нужно и переименование зависит от слова-разделителя.
        if replacement_dict and split_word in replacement_dict:
            new_row = [replacement_dict[split_word] if split_word in item else item for item in new_row]

        return new_row
    
    # Общий прогресс можно убрать. Просто у меня файл достаточно большой и это занимает пару минут. Так чисто, декоративное.
    ##########vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv##############
    # Общий прогресс прогресс бара от 0 до 100% и Инициализация прогресс бара с настройками отображения.
    total_progress = 100
    with tqdm_notebook(total=total_progress, desc=f"Cell progress: ", bar_format='{desc}: {percentage:.0f}%|{bar}| {n:.0f}/{total_fmt} [Elapsed: {elapsed} m/s, Remaining: {remaining} m/s]') as pbar:
        cumulative_progress = 0 # Чтобы прогресс бар работал, как надо и завершался всегда на 100%.
    ##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############
        
        # Здесь сам цикл, в котором мы перебираем каждое слово и передаём его в каждую строку, которая является списком.))
        # И обрабатываем это слово с помощью функции transform для каждой строки.
        for i, word in enumerate(words_to_transform):
            tags_column = tags_column.apply(transform, split_word=word, list_of_exceptions=list_of_exceptions, replacement_dict=replacement_dict)
            
    ##########vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv##############        
            # Рассчитываем размер шага на основе оставшегося прогресса и оставшихся итераций.
            remaining_iterations = len(words_to_transform) - i # вот тут пригодился индекс i
            remaining_progress = total_progress - cumulative_progress
            step_size = remaining_progress / remaining_iterations
            
            # Обновляем общий прогресс.
            cumulative_progress += step_size
            pbar.update(step_size)
    ##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############

    return tags_column

[to contents](#contents)

<h3 id='multiplemodelstraining'>Функция для тренировки нескольких моделей, используя несколько алгоритмов ML за 1 заход</h3>

In [ ]:
def multiple_models_training(df, target_feature, list_of_ml_algs, params_dict=False, random_state=42, test_size=0.25):
    print(f'{Fore.RED}КОРОЛЬ АРТУР НА НАС НАПАЛИ!!!{Fore.RESET}') # https://www.youtube.com/watch?v=-XuyTXj_5F8&t=6s хз, чёт вспомнилось...
    """ Freeware by Akialema.
    Сие ваяние было вдохновлено... В телеграмме периодически запускают эвент типа мол: "попробуйте себя в роли датасаентиста",
    который длится 3 дня. И на третий день предлагалось сделать вот это вот, что происходит в проекте-3 и что происходит в этой функции: 
    х-трэйны, у-трэйны, тэсты, предсказания, кто там на что делится.. тыща параметров и т.д. И финальным заданием было попробовать 
    перебрать несколько алгоритмов, чтобы получить наименьшее некое число. И вот в тот момент я понял, что для этого дела нужен цикл.
    С графическим представлением. Потому что не даром их так много. А потом я посмотрел прекрасную работу Андрея Волкова и такой: 
    вот же она, эта картинка.) И чтоб не плодить простыню х-трэйнов-у-трэйнов, её можно просто обернуть в цикл и в функцию.
    Я сначала делал график прям в функции, чтоб такой ждёшь, ждёшь выполнения ячейки и хочется какой-то интерактивности прогресса 
    с последующим финальным представлением графика сравнения МАРЕ на сладкое. Но потом понял, что лучше делать картиночку отдельной ячейкой.
    Короче кайфанул и доволен. Да и я хз, может есть уже реализация подобного, я ж новичок. Потому быстро накалякал своё. Вообще не понимаю,
    что там происходит и правильно ли. Но работает. Я просто хорошо задаю вопросы чату гпт.))) 
    Забирайте, если понравилось.)
    
    P.S. функция не идеальна. Я не знаю названия всех алгоритмов. Какие чат гпт сказал, что топ, те и тут. Например функция явно не для 
    AutoML, там какая-то длинная реализация.. в работе Андрея.)) ыы Сам я пока не рискну.) 
    Явно я эту функцию ещё дополню. Или пойму, что это пустая трата времени. 
    P.S.2 ЕСЛИ ВЫ ЗНАЕТЕ, КАК СДЕЛАТЬ ПЛАВНЫЙ ПРОГРЕСС БАР ПО ЯЧЕЙКЕ - РАССКАЖИТЕ МНЕ. Пожалуйста. <^-------- Можно не читать.
    
    Функция проведения нескольких (или одной) тренировок моделей.
    
    ---> На вход подаём итоговый датафрейм, в котором только числа и нет нанок, 
         целевой признак,
         список названий алгоритмов для тренировки,
         и словарь с параметрами для алгоритмов.

    Args:
        df (pd.Dataframe): итоговый дф.
        target_feature (str): целевой признак.
        list_of_ml_algs (list): список Агрессоров, каждый из них в str формате.)) (агрессоров может быть как 1, так и сколько душе угодно, но важно, чтобы они соответствовали структуре "инит класса -> тренировка -> предсказание").
        params_dict (dict, optional): словарь вида {alg: [{параметры алгоритма}, {параметры фита}]}, по которому создаётся датафрейм с параметрами. Первый внутренний словарь для параметров алгоритма, второй - для фита. Defaults to False.
        random_state (int, optional): я хз, что это, я так до сих пор и не прочувствовал np.random.seed. Defaults to 42.
        test_size (float, optional): размер тестовой выборки. Defaults to 0.25.

    Returns:
        pd.Dataframe: На выход получаем датафрейм с натренированными моделями, числом МАРЕ и чем-нибудь ещё.
        
    Пример работы. Нужно скопировать код ниже и вставить в ячейку после функции.
    ------------------------------------------------------------------------------------------------------
    # Здесь нужно выбрать, какими алгоритмами будем гонять числа.)
    list_of_ml_algs = ['RandomForestRegressor', 'ExtraTreesRegressor', 'BaggingRegressor', 'GradientBoostingRegressor', 'CatBoostRegressor', 'AdaBoostRegressor', 'DecisionTreeRegressor']
    
    # Параметры для каждого алгоритма. Порядок не важен, как и указание параметров для всех алгоритмов. 
    # Можно указать параметры только для тех, для которых нужно, остальные отработают по своим дефолтным параметрам.
    # Важна структура {alg: [{параметры алгоритма}, {параметры фита}]}, иначе можно хлебнуть горя.)) Индекс-столбец-столбец.
    # А можно и не делать этот словарь вовсе. Алгоритмы отработают по своим настройкам по-умолчанию.
    params_dict = {
            'RandomForestRegressor': [{'n_estimators': 10, 'n_jobs': -1, 'random_state': 42}, {}],
            'ExtraTreesRegressor': [{'n_estimators': 10, 'max_depth': 3}, {}],
            'BaggingRegressor': [{'n_estimators': 10}, {}],
            'GradientBoostingRegressor': [{'n_estimators': 10, 'learning_rate': .1}, {}],
            'CatBoostRegressor': [{'iterations': 10000, 'learning_rate': .1, 'depth': 3, 'verbose': False}, {}],
            'AdaBoostRegressor': [{'n_estimators': 10, 'learning_rate': 1.0}, {}],
            'DecisionTreeRegressor': [{}, {}]
    } # везде эстиматоры 10, чтоб оно отработало быстро и показало работу. 
    
    # Здесь нужно вставить свой end_df, свой 'целевой_признак', решить, какими алгоритмами будем баловаться и с какими параметрами.)
    any_var_name = multiple_models_training(end_df, 'reviewer_score', list_of_ml_algs=list_of_ml_algs, params_dict=params_dict)
    
    any_var_name
    ------------------------------------------------------------------------------------------------------
    
    Нужно быть внимательным/ной, чтоб всё сработало, как надо. Я вроде всё расписал.
    """
    
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from colorama import Fore, Back, Style # Чтоб аутпут был цветастый.
    from tqdm.notebook import tqdm
    import pandas as pd
    import time
    
    # Если мы передаём параметры словарём извне.
    if params_dict:
        params_dict_func = {alg: [{}, {}] for alg in list_of_ml_algs} # Создаётся словарь следующей формы: {alg: [{}, {}]}, для каждого элемента списка с названиями алгоритмов.
        params_dict_func.update(params_dict) # Обновление словаря словарём извне.
        params_df = pd.DataFrame.from_dict(params_dict_func, orient='index', columns=['alg_params', 'fit_params']) # ДФ по словарю.
    # Если мы не передаём параметры в функцию, то все параметры будут по умолчанию для каждого алгоритма.
    else:
        params_df = pd.DataFrame([[{},{}]], index=list_of_ml_algs, columns=['alg_params', 'fit_params'])
    
    X = df.drop(columns=[target_feature], axis=1) # дропаем целевой признак из икса.
    y = df[target_feature] # и помещаем его в игрек.
    
    # Эти штуки.. Я надеюсь они в правильном порядке стоят. Вроде бы у всех так.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    list_of_algs_output = [] # список по которому будем формировать выходящий датафрейм.
    
    total_progress = 100  # Общий прогресс прогресс бара от 0 до 100%
    
    # Инициализация прогресс бара с настройками отображения.
    with tqdm(total=total_progress, desc=f"Cell progress: ", bar_format='{desc}: {percentage:.0f}%|{bar}| {n:.0f}/{total_fmt} [Elapsed: {elapsed} m/s]') as pbar:
        
        cumulative_progress = 0 # Чтобы прогресс бар работал, как надо и завершался всегда на 100.
        
        # Цикл по списку алгоритмов.
        for i, alg in enumerate(list_of_ml_algs): # i пригодился в вычислении шага прогрессбара.
            
            start_alg_time = time.time() # Время начала работы обучения модели.
            
            # Инит алгоритма через globals, т.к. название алгоритма == str. (можно через eval(), но гпт чату не понравилась эта идея) 
            # Параметры == параметры столбца 'class_params' для строки с именем alg.
            model = globals()[alg](**params_df.loc[alg, 'alg_params']) 
            
            model.fit(X_train, y_train, **params_df.loc[alg, 'fit_params']) # Здесь сама тренировка. Параметры из столбца 'fit_params'.
            
            current_time = time.time() # Время окончания работы алгоритма.
            
            y_pred = model.predict(X_test) # Предсказание.
            
            # Делаем МАПЕ.
            mape = metrics.mean_absolute_percentage_error(y_test, y_pred) * 100
            # Я не знаю, что это и для чего это здесь. Просто добавил, чтобы убедиться, что в вывод можно ещё чего-нибудь прикрутить.
            # Соответственно, нужно обновлять аппенд и формирование столбцов итогового дф.
            accuracy = model.score(X_test, y_test) 
            
            list_of_algs_output.append([alg, mape, accuracy, model]) # Добавляем всю эту петрушку в список, по которому будет формироваться таблица.
            
            ##########################################################################################################################
            
            # Вычисление времени для принта ниже. Типа вот, мол, модель обучилась, поздравляю.
            execution_time_alg = current_time - start_alg_time
            # Можно прикрутить, что-то другое для принта.
            print(f"Model training completed! With: {str(round(execution_time_alg, 2)) + ' sec.' if execution_time_alg < 60 else str(round(execution_time_alg/60, 2)) + ' min.'} Alg: {Fore.CYAN}{alg}{Fore.RESET}.") # чтоб секунды были секундами, а минуты - минутами.
            time.sleep(0.2) # Сон я добавил, потому что сначала прогресс бар отрабатывает, а потом принт. А надо наоборот.)
            
            # Рассчитываем размер шага на основе оставшегося прогресса и оставшихся итераций.
            remaining_iterations = len(list_of_ml_algs) - i # вот тут пригодился индекс i
            remaining_progress = total_progress - cumulative_progress
            step_size = remaining_progress / remaining_iterations
            
            # Обновляем общий прогресс. Все округления происходят при инициализации прогрессбара.
            cumulative_progress += step_size
            pbar.update(step_size)
        
    func_df = pd.DataFrame(list_of_algs_output, columns=['alg_name', 'mape_percent', 'accuracy', 'model']).sort_values(by='mape_percent').reset_index(drop=True) # ДФ-чик 
    
    print(f'{Fore.GREEN}Mission accomplished{Fore.RESET}.')
    # Возвращаем дф, в котором мапе и сама модель. 
    return func_df

[to contents](#contents)

<h3 id='vadersentanalsec'>Функция для формирования датафрейма с определением тональности текста по алгоритму Вейдера.</h3>

In [ ]:
def vader_sentiment_analyser(df, targ_col):
    """Функция для формирования датафрейма с определением тональности текста по алгоритму Вейдера.
    
    Args:
        df (pd.DataFrame): incoming df
        targ_col (str): target column

    Returns:
        pd.DataFrame: df with neg, neu, pos, compound from Vader.
        
        looks like:
        __|targ_col_neg|targ_col_neu|targ_col_pos|targ_col_comp|
        0 |         0.1|         0.8|         0.2|         0.64|
        etc.
    """
    # Импорт необходимых библиотек. Библиотеки лучше выносить за рамки функции.
    import pandas as pd
    from tqdm.notebook import tqdm
    import nltk
    from nltk.sentiment import SentimentIntensityAnalyzer
    nltk.download('vader_lexicon')
    
    # Инит класса.
    sia = SentimentIntensityAnalyzer()
    # Словарь для записи результатов.
    res = {}
    
    total_progress = 100
    with tqdm(total=total_progress, desc=f"Cell progress: ", bar_format='{desc}: {percentage:.0f}%|{bar}| {n:.0f}/{total_fmt} [Elapsed: {elapsed} m/s, Remaining: {remaining} m/s]') as pbar:
        cumulative_progress = 0 # Чтобы прогресс бар работал, как надо и завершался всегда на 100.
    ##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############
        
        # Здесь сам цикл, в котором мы перебираем каждое слово и передаём его в каждую строку, которая является списком.))
        # И обрабатываем это слово с помощью функции transform для каждой строки.
        for i, text in enumerate(df[targ_col]):
            res[i] = sia.polarity_scores(text)
            
    ##########vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv##############        
            # Рассчитываем размер шага на основе оставшегося прогресса и оставшихся итераций.
            remaining_iterations = len(df) - i # вот тут пригодился индекс i
            remaining_progress = total_progress - cumulative_progress
            step_size = remaining_progress / remaining_iterations
            
            # Обновляем внутренний и общий прогресс.
            cumulative_progress += step_size
            pbar.update(step_size)
    ##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############
    # Переименование столбцов.
    new_columns = {col: f"{targ_col}_{col}" for col in res[0].keys()}
    # Возврат дф-ки с решейпом и переименованием столбцов.
    return pd.DataFrame(res).T.rename(columns=new_columns)

[to contents](#contents)

<h3 id='vadersentanalmain'>Функция для формирования датафрейма с определением тональности текста по алгоритму Вейдера (основная).</h3>

In [ ]:
def vader_sentiment_analysis(df, targ_col):
    """Функция для формирования датафрейма с определением тональности текста по алгоритму Вейдера.
    
    Args:
        df (pd.DataFrame): incoming df
        targ_col (str): target column

    Returns:
        pd.DataFrame: df with neg, neu, pos, compound from Vader.
        
        looks like:
        __|targ_col_neg|targ_col_neu|targ_col_pos|targ_col_comp|
        0 |         0.1|         0.8|         0.2|         0.64|
        etc.
    """
    
    # Импорт необходимых библиотек. Библиотеки лучше выносить за рамки функции.
    import pandas as pd
    from tqdm.notebook import tqdm
    import nltk
    from nltk.sentiment import SentimentIntensityAnalyzer
    nltk.download('vader_lexicon')
    
    # Инит класса.
    sia = SentimentIntensityAnalyzer()
    # Словарь для записи результатов.
    res = {}
    
    # Цикл по строкам датафрейма и запись анализа настроения в словарь.
    for i, text in enumerate(tqdm(df[targ_col], total=len(df))):
        res[i] = sia.polarity_scores(text)
    
    # Переименование столбцов.
    new_columns = {col: f"{targ_col}_{col}" for col in res[0].keys()}
    # Возврат дф-ки с решейпом и переименованием столбцов.
    return pd.DataFrame(res).T.rename(columns=new_columns)

[to contents](#contents)

<h3 id='robertapolarity'>Функция с алгоритмом Роберты на основе отзывов в твиттере.</h3>

In [ ]:
def roberta_polarity_scores(df, targ_col):
    """Функция для формирования датафрейма с определением тональности текста по алгоритму Роберты.
    
    Args:
        df (pd.DataFrame): incoming df
        targ_col (str): target column

    Returns:
        pd.DataFrame: df with neg, neu, pos, compound from Vader.
        
        looks like:
        __|targ_col_neg|targ_col_neu|targ_col_pos|
        0 |         0.1|         0.8|         0.2|
        etc.
    """
    # Подгрузка библиотек. Библиотеки лучше вынести за пределы функции.
    from transformers import AutoTokenizer
    from transformers import AutoModelForSequenceClassification
    from scipy.special import softmax
    
    # Инит классов, токенизеров.
    MODEL = f'cardiffnlp/twitter-roberta-base-sentiment'
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    
    # Словарь, по которому формируется выходящий дф.
    res = {}
    # Цикл по строкам столбца.
    for i, text in enumerate(tqdm(df[targ_col], total=len(df))):
      # Трай - эксепт здесь для того, что некоторые строки слишком длинные и алгоритм роберты с ними не справляется и выдаёт ошибку.
      try: 
        # Всё, что ниже можно написать в 1 строку, но го в 4.
        encoded_text = tokenizer(text, return_tensors='pt') # Тут токенизация слов.
        output = model(**encoded_text) # Тут я не помню, какой аутпут был, если интересно - ниже в одну строку и можно обрубить, и посмотреть на каждый этап.
        scores = softmax(output[0][0].detach().numpy()) # Здесь происходит экспоненцирование (вот такое вот слово) чисел с помощью softmax.
        res[i] = {'roberta_neg': scores[0], 'roberta_neu': scores[1], 'roberta_pos': scores[2]} # Ну и запись в словарь.
      except RuntimeError:
        print(f'Broke for id: {i}') # если ошибка, то выводим номер строки.

    # Переименование столбцов.
    new_columns = {col: f"{targ_col}_{col}" for col in res[0].keys()}
    # Возврат дф-ки с решейпом и переименованием столбцов.
    return pd.DataFrame(res).T.rename(columns=new_columns)

# Эта строчка выведет настроение конкретной строки текста, т.е. вместо 'example' нужно вставить 'любой текст'.
#print(softmax(AutoModelForSequenceClassification.from_pretrained(f'cardiffnlp/twitter-roberta-base-sentiment')(**AutoTokenizer.from_pretrained(f'cardiffnlp/twitter-roberta-base-sentiment')('example', return_tensors='pt'))[0][0].detach().numpy())) # вместо 'example' любой текст

[to contents](#contents)

<h3 id='optunafunc'>Optuna (тестовая)</h3>

In [ ]:
def optuna_mult(trial, X_train, y_train, param_dict, estimator, metric, cross_val=None):
    
    from functools import partial # чтобы вызывать функции с доп параметрами
    import optuna
    optuna.logging.set_verbosity(optuna.logging.WARNING) # INFO, WARNING, CRITICAL, ERROR
    from sklearn.model_selection import KFold, cross_val_score
    from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
    
    # блок сетки параметров
    func_dict = {}
    
    for i, j in param_dict.items():
        
        if isinstance(j, dict):
            if 'low' in j:
                if isinstance(j['low'], float):
                    func_dict[i] = trial.suggest_float(**j)
                else:
                    func_dict[i] = trial.suggest_int(**j)
            else:
                func_dict[i] = trial.suggest_categorical(**j)
        else:
            func_dict[i] = j
    
    # имя для метрики (можно и нужно расширить эту хрень)
    func_score_metric = ''
    if metric in ['mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_error']:
        func_score_metric = 'neg_' + metric

    # создаем модель
    model = globals()[estimator](**func_dict)
    
    if cross_val is None:
        # обучаем модель
        model.fit(X_train, y_train)
        score = globals()[metric](y_train, model.predict(X_train))
    else:
        # задаём параметры кросс-валидации (стратифицированная 5-фолдовая с перемешиванием)
        kf = globals()[cross_val](n_splits=5, shuffle=True, random_state=func_dict['random_state'])
        # проводим кросс-валидацию  
        score = -cross_val_score(estimator=model, X=X_train, y=y_train, 
                                 scoring=func_score_metric, cv=kf, n_jobs=-1).mean()
    return score

############################### ниже применение

param_dict_lasso = {'alpha': {'name': 'alpha', 'low': 0.05, 'high': 1, 'step': 0.05},
                    'max_iter': {'name': 'max_iter', 'low': 1000, 'high': 10000, 'step': 1000},
                    'tol': {'name': 'tol', 'low': 0.0001, 'high': 0.001, 'step': 0.00005},
                    'random_state': 42}

study_lasso = optuna.create_study(study_name="Lasso", direction="minimize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_lasso.optimize(partial(optuna_mult, 
                             X_train=mid_df_poly, y_train=y, 
                             param_dict=param_dict_lasso,
                             estimator='Lasso',
                             metric='mean_squared_error',
                             cross_val='KFold'), n_trials=40, show_progress_bar=True)

# выводим результаты на обучающей выборке
print(f"Наилучшие значения гиперпараметров {study_lasso.best_params}")
print(f"MAE на обучающем наборе: {study_lasso.best_value:.2f}")

[to contents](#contents)

<h3 id='func_research'>Исследование функции для одной переменной.</h3> сырая

In [ ]:
def my_function_research(f, ylimits=[-10, 10], xlimits=[-10, 10], x_vals=np.linspace(-10, 10, 100), show_extr=False):
    """my_function_research 

    Функция, выводящая сводку для простых функций. Из серии "исследование функции"

    Args:
        f (formula): x**3 + 4*x**2 - 4*x - 16 like
        ylimits (list, optional): list of limits to correct visualisation. Defaults to [-10, 10].
        xlimits (list, optional): list of limits to correct visualisation. Defaults to [-10, 10].
        x_vals (_type_, optional): range of x-nums to.. plot. Defaults to np.linspace(-10, 10, 100).
        show_extr (bool, optional): show extremums or not. Try. Defaults to False.

    Returns:
        none: this is print func
    """
    import pandas as pd
    import matplotlib.pyplot as plt
    import sympy
    from IPython.display import Math, display
    
    def find_extremum_coordinates(df, searchnum):
        
        """ 
        Функция для печати экстремум точек на графике. Функция сырая, потому, если будет ошибка, связанная с вычислением этих точек, то просто
        show_extr=False и тогда эта функция не будет отрабатывать.
        
        df - это таблица о двух столбцах = решение функции для каждого икса из x_vals.
        В столбце 0 находятся иксы, а в столбце 1 - игреки.
        searchnum - это... корни первой производной.
        
        """
    
        func_df = pd.DataFrame()
        second_der_vals = [sympy.diff(f, x, 2).subs(x, i).evalf() for i in sympy.solve(sympy.diff(f, x))]
        
        # Бежим по корням.
        for i, j in enumerate(searchnum):
            
            func_series = df.iloc[:, 0].copy()
            
            # Для каждого корня мы ищем в цикле while диапазон чисел (до 10 штук), которые приблизительно равны каждому корню.
            while len(func_series) > 10:
                median = func_series.median()
                
                # Здесь находится костыль. Т.е. у линспейса от -10 до 10 медиана = 0 
                # и если корень производной = 0, то берем дапазон чисел вокруг нуля в кол-ве 9 шт. (-4, +5)
                if j == 0:
                    func_series = func_series.iloc[len(func_series)//2 - 4:len(func_series)//2 + 5]
                
                elif j < median:
                    func_series = func_series.loc[func_series <= median]
                    
                elif j > median:
                    func_series = func_series.loc[func_series >= median]
            
            # Далее смотрим, если значение второй производной < 0 - значит ищем максимальное значение, которое является экстремумом,
            # И то же самое, если взначение второй производной > 0: ищем минимальное.
            # А вот для = 0 снова костыль. Я беру медиану. Поэтому в строке выше (-4, +5), чтоб было нечётное кол-во.
            func_series_df = df.iloc[func_series.index]
            len_of_pos = len(func_series_df.loc[func_series_df[1] > 0])
            len_of_neg = len(func_series_df.loc[func_series_df[1] < 0])
            
            # Ниже замес следующий:
            # Если вторая производная < 0, то берем максимальное значение по у, если нет, то наоборот, если 0, то медиану берем (костыль)
            # Далее идёт.. в столбце могут быть не только одни + или они -, но и +/- и тогда сравниваем длины + и -, 
            # и берем нужный максимум или минимум по тому списку, который больше.
            # Т.к. там может быть резкий скачок, если например внизу парабола, которая смотрит вниз, а через 0 идёт кубическая парабола.
            # И там, где заканчивается парабола начинается куб.парабола. Ну типа они обе стремятся, скажем, к вертикальной асимптоте -1.
            if second_der_vals[i] < 0: 
                if all(func_series_df[1] > 0) or all(func_series_df[1] < 0):
                    func_df[i] = func_series_df.loc[func_series_df[1].idxmax()]
                else:
                    if len_of_pos > len_of_neg:
                        func_df[i] = func_series_df.loc[func_series_df[1] > 0].loc[func_series_df.loc[func_series_df[1] > 0, 1].idxmax()]
                    else:
                        func_df[i] = func_series_df.loc[func_series_df[1] < 0].loc[func_series_df.loc[func_series_df[1] < 0, 1].idxmax()]
            elif second_der_vals[i] > 0:
                if all(func_series_df[1] > 0) or all(func_series_df[1] < 0):
                    func_df[i] = func_series_df.loc[func_series_df[1].idxmin()]
                else:
                    if len_of_pos > len_of_neg:
                        func_df[i] = func_series_df.loc[func_series_df[1] > 0].loc[func_series_df.loc[func_series_df[1] > 0, 1].idxmin()]
                    else:
                        func_df[i] = func_series_df.loc[func_series_df[1] < 0].loc[func_series_df.loc[func_series_df[1] < 0, 1].idxmin()]
            elif second_der_vals[i] == 0:
                func_df[i] = func_series_df.loc[func_series_df[1] == func_series_df[1].median()].iloc[0, :] # там получается дф, потому берем строку
        return func_df
    
    x = sympy.Symbol('x')
    
    first_derivative = sympy.diff(f, x)
    display(Math(f"Первая \ производная: {sympy.latex(first_derivative)}"))
    
    roots = sympy.solve(first_derivative, x)
    display(Math(f"Корень/ни \ первой \ производной: {sympy.latex(roots)}"))
    
    second_derivative = sympy.diff(first_derivative, x)
    display(Math(f"Вторая \ производная: {sympy.latex(second_derivative)}"))
    
    values_of_second_der = [second_derivative.subs(x, i).evalf() for i in roots]
    display(Math(f"Значение/я \ второй \ производной \ для \ корней \ первой: {sympy.latex(values_of_second_der)}"))
    
##################### Дальше график
    x_vals = x_vals
    vals = [f.subs(x, i) for i in x_vals]
    plot_df = pd.DataFrame([x_vals, vals]).T.astype(float)
    
    plt.figure(figsize=(4,4))
    
    plt.axvline(0, color='black', linewidth=0.5, linestyle='--')
    plt.axhline(0, color='black', linewidth=0.5, linestyle='--')
    
    # Условие можно продолжить, там ещё лог, син, кос, тан и т.д.
    if isinstance(sympy.denom(f), sympy.Symbol) or isinstance(sympy.denom(f), sympy.Add) or isinstance(sympy.denom(f), sympy.Pow):
        # Ищем, в каких числах знаменатель будет 0, условие для комплексных чисел.
        crit_nums = [i.evalf() for i in sympy.solve(sympy.denom(f), x) if not isinstance(i, sympy.core.add.Add)]
        # Критических чисел может быть много и для "многих" - условие.
        if len(crit_nums) > 1:
            # Сначала рисуем крайние справа и слева.
            for i in crit_nums:
                if i == min(crit_nums):
                    plt.plot(plot_df.loc[plot_df[0] < i, 0], plot_df.loc[plot_df[0] < i, 1])
                elif i == max(crit_nums):
                    plt.plot(plot_df.loc[plot_df[0] > i, 0], plot_df.loc[plot_df[0] > i, 1])
            # А потом рисуем промежутки.
            # Например у нас списко крит_нумов = [1,2,3,4,5]
            # В цикле перебираем: [i, j] [1,2], [2,3], [3,4], [4,5] ну и меньше-больше.
            for i, j in zip(crit_nums, crit_nums[1:]):
                
                plt.plot(plot_df.loc[(plot_df[0] > i) & (plot_df[0] < j), 0], plot_df.loc[(plot_df[0] > i) & (plot_df[0] < j), 1])
        # Иначе, если крит_нум - это 1 число 
        else:
            plt.plot(plot_df.loc[plot_df[0] < crit_nums[0], 0], plot_df.loc[plot_df[0] < crit_nums[0], 1])
            plt.plot(plot_df.loc[plot_df[0] > crit_nums[0], 0], plot_df.loc[plot_df[0] > crit_nums[0], 1])
    # Иначе знаменатель нормальный и просто рисуем.)
    else:
        plt.plot(x_vals, vals)
    # Если хотим видеть метки экстремумов, типа пик выпуклости или впуклости
    if show_extr:
        
        roots_4_coord_df = [float(i.evalf()) for i in roots]
        coord_df = find_extremum_coordinates(df=plot_df, searchnum=roots_4_coord_df)
        
        # Добавляем аннотацию с метками точек экстремумов на оси X
        for i, j in enumerate(roots):
            x_coord = coord_df.iloc[0, i] # координаты точки
            y_coord = coord_df.iloc[1, i]
            plt.annotate(f"Extremum: ${sympy.latex(j)}$",
                         xy=(x_coord, y_coord), 
                         xytext=(x_coord+1, y_coord),  # Смещаем текст немного вправо
                         backgroundcolor='w',
                         arrowprops=dict(facecolor='black', shrink=0.03, width=0.7, headwidth=4, headlength=10))
    
    plt.xticks(list(range(xlimits[0], xlimits[1]+1)), fontsize=6)
    plt.yticks(list(range(ylimits[0], ylimits[1]+1)), fontsize=6)
    
    plt.title('График для входящей функции:')
    plt.ylim(ylimits)
    plt.xlim(xlimits)
    plt.show()
    
    #return coord_df
    #return pd.DataFrame([[i.evalf() for i in roots], values_of_second_der], index=['Корни 1 производной', 'Значения 2 для корней первой']).T

[to contents](#contents)

<h3 id='func_research_xyz'>Исследование функции для двух, трёх переменных.</h3>

In [ ]:
def extremums_of_mult_vars_func(f, return_='df', range_of_nums_for_plot=np.linspace(-5, 5, 100)):
    
    import re
    import numpy as np
    import pandas as pd
    import sympy as sp
    
    # Тут я пытался оформить отображение латех формул в ячейках датафрейма... не получилось, оставлю
    def render_latex(row):
        return [f"$ {val} $" for val in row]
    
    # эта функция делает из x**2 + y**2 -> x<sup>2</sup> + y<sup>2</sup> (это пример)
    # короче костыль. я хз как печатать в плотли формулы.(
    def latex_to_html_converter(string):
    
        list_of_crit_symbols = ['(', ')', '+', '-', '/']    
        func_str = string
        func_str = ''.join([' ' + i + ' ' if i in list_of_crit_symbols else i for i in func_str])
        
        func_str = func_str.replace('**', '<sup>').replace('*', '')
        
        if 'sqrt' in func_str:
            func_str = func_str.replace('sqrt', '\u221A')
        
        if '<sup>' in func_str:
            
            func_str = func_str.split(' ')
            
            for i, j in enumerate(func_str):
                if '<sup>' in j:
                    func_str[i] = func_str[i] + '</sup>'
            func_str = ' '.join(func_str)
        
        return func_str.replace(' ', '')
    
    # отсортированный список переменных ['x', 'y', ...]
    vars = sorted(list({i for i in re.sub(r'sqrt|exp|log|sin|cos', '', str(f)) if i.isalpha()}))
    
    display(Math(f"Incoming \ func: {sp.latex(f)}"))
    display(*[Math(f"f_{i}': {sp.latex(sp.diff(f, i))}") for i in vars])
    display(Math(f"f_{{xx}}'': {sp.latex(sp.diff(f, vars[0], 2))}"))
    
    # решение для первой производной
    critical_points = sp.solve([sp.diff(f, i) for i in vars], vars)
    
    # проверка, если там только одно решение и возвращается словарь
    # т.к. у меня функция заточена под список кортежей
    if isinstance(critical_points, dict):
        critical_points = [tuple(critical_points.values())]
    
    # тут нужна проверка на комплексные числа
    if not all(j.is_real for i in critical_points for j in i):
        print('There is a complex numbers in critical points')
        critical_points = [pnt for pnt in critical_points if all(s.is_real for s in pnt)]
    
    # словарь вторых производных
    second_der_dict = {'f_'+i+j: sp.diff(sp.diff(f, i), j) for i in vars for j in vars} #{'f_'+''.join(i): diff(diff(f, i[0]), i[1]) for i in itertools.product(vars, repeat=2)}
    
    # ниже два условия: если переменных 2 и если 3, и формулы для них
    # когда переменных две - в дополнение строится график входящей функции
    if len(vars) == 2:
        
        Hessian = second_der_dict['f_xx'] * second_der_dict['f_yy'] - second_der_dict['f_xy']**2
        
        import numpy as np
        import plotly.graph_objs as go
        
        f_func = sp.lambdify((vars), f, 'numpy')

        # Создание сетки значений
        X = range_of_nums_for_plot
        Y = range_of_nums_for_plot
        X, Y = np.meshgrid(X, Y)
        Z = f_func(X, Y)
        
        # Создание 3D поверхности
        surface = go.Surface(z=Z, x=X, y=Y, colorscale='Viridis')
        
        # Настройка макета. костыль в title. плотли не умеет отображать латех, но умеет хтмл, потому sup-ы. см функцию.
        layout = go.Layout(title='Incoming func: ' + latex_to_html_converter(str(f)),#'Incoming func: ' + ' '.join([i + '</sup>' if 'sup' in i else i for i in str(f).replace('**', '<sup>').replace('*', '').split()]),#'&mu;g/m<sup>2</sup>',
                           scene = dict(xaxis_title='X',
                                        yaxis_title='Y',
                                        zaxis_title='Z'),
                           width=600,
                           height=400)
        
        # Построение фигуры
        fig = go.Figure(data=[surface], layout=layout)
        
        # Отображение графика
        fig.show()
    
    if len(vars) == 3:
        
        sdd = second_der_dict
        Hessian = sdd['f_xx']*sdd['f_yy']*sdd['f_zz'] + 2*sdd['f_xy']*sdd['f_xz']*sdd['f_yz'] - sdd['f_xx']*sdd['f_yz']**2 - sdd['f_yy']*sdd['f_xz']**2 - sdd['f_zz']*sdd['f_xy']**2
    
    if return_ == 'df':
        # Ну и тут 3 столбца: крит точки, решение для второй производной по хх, т.к. это первый минор и решение уравнения гессиана
        return pd.DataFrame([[[float(k.evalf()) for k in point], float(second_der_dict['f_xx'].subs({var: point[i] for i,var in enumerate(vars)}).evalf()), float(Hessian.subs({var: point[i] for i,var in enumerate(vars)}).evalf())] for point in critical_points], columns=['crit_nums', 'first_minor', 'hessian_det'])#.style.format(na_rep='', escape='latex').apply(render_latex)
    else:
        return pd.DataFrame(np.array(list(second_der_dict.values())).reshape(len(second_der_dict)//len(vars), len(second_der_dict)//len(vars)))

[to contents](#contents)

<h3 id='gradfxinteractive'>Интерактивный график градиентного спуска для функции одной переменной.</h3>

In [ ]:
def plot_grad_fx(f, df, n_iter=20, start_point=0, step=0.1, mn=100, x_range=np.arange(0, 5.0, 0.1), figsize=(6, 6)):
    
    # https://www.youtube.com/watch?v=OKeZEbJgQKc видео с этим кодом
    """ Пока не оформлено

    Пример работы:
    
    def f(x):
    return x*x - 5*x + 5

    def df(x):
        return 2*x - 5
    
    plot_grad_fx(f, df)
    """
    
    import time
    import matplotlib.pyplot as plt
    import numpy as np
    from IPython.display import clear_output
    
    f_plt = [f(x) for x in x_range]
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.grid(True)
    
    ax.plot(x_range, f_plt)
    point = ax.scatter(start_point, f(start_point), c='red')
    
    for i in range(n_iter):
        # Здесь изменяется шаг сходимости. С большего к меньшему с константой mn, 
        # т.е. на последней итерации шаг будет 1/100 = 0.01
        step = 1 / (np.min((i+1, mn)))
        # np.sign возвращает знак, + или -
        start_point -= step * np.sign(df(start_point))
        
        # Обновляем положение точки
        point.set_offsets([start_point, f(start_point)])
        
        # Обновляем график для интерактивного отображения
        clear_output(wait=True)
        display(fig)
        time.sleep(0.05)
    
    print(start_point)
    plt.close(fig)

[to contents](#contents)

<h3 id='strongcolssearcher'>Функция для автоматического поиска сильных столбцов относительно целевого столбца (тестовая).</h3>

In [ ]:
def strong_cols_auto_searcher(df, list_of_cat_cols, list_of_num_cols, list_of_agg_func, targ_col, threshold_number=4000):
    import pandas as pd
    import numpy as np
    from sklearn.feature_selection import f_classif
    from tqdm.notebook import tqdm
    
    """strong_cols_auto_searcher:

    Функция, которая ищет наиболее значимые столбцы, для предсказания целевого столбца.
    т.е. по очереди происходит группировка всех названий столбцов из списка list_of_cat_cols с столбцами из списка list_of_num_cols
    с агрегацией функций из списка list_of_agg_func и сразу происходит отсев по пороговому числу threshold_number.
    Ну и запись в выходящий датафрейм.

    Args:
        df (pd.DataFrame): incoming df.
        list_of_cat_cols (list): [['col1'], ['col2'], ['col3', 'col4']] list with lists inside with str in this lists.
        list_of_num_cols (list): ['col1', 'col2', 'col3'] just list with str elements.
        list_of_agg_func (list): ['mean', 'count', 'nunique', 'sum'] list with any func name.
        targ_col (str): 'col' - any col u like.
        threshold_number (int, optional): minimum for f_classif. if < then we dont need this column. Defaults to 4000.

    Returns:
        pd.DataFrame: df with data about what and by what was groupped with what function for each column at the last row of each column. 
    """
    # Копия для основной таблицы.
    main_df = df.fillna(0).copy()
    # Номер последней строки, куда потом будет происходить запись что с чем группируем и по какой функции.
    num_for_new_row = main_df.shape[0]
    
    # Пустой итоговый дф.
    func_dict = {}
    annot_dict = {}
    
    # Цикл по тому, по чему группируем. С добавлением прогресс бара.
    for i in tqdm(list_of_cat_cols, total=len(list_of_cat_cols)):
        # Цикл по тому, что группируем. 
        for j in list_of_num_cols:
            
            # Если столбец есть и там, и там, то пропускаем такую группировку.
            if j in i:
                continue
            
            # Если тип столбца для группировки О (например там столбец с строками)
            if np.issubdtype(main_df[j], 'object'):
                # То применяем только count, т.к. mean не проканает, а sum - бессмысленен.
                func_series = main_df.groupby(i)[j].transform('count') ############ здесь слабое место, я пока не придумал, как его обыграть.
                # Присваиваем этому срезу новое название.
                func_series.name = 'strong_column_' + str(len(func_dict))
                # Далее происходит запись в словари. Записывается сам сериес и данные по сериесу, каждая запись в свой словарь.
                func_dict[func_series.name] = func_series.to_dict()
                annot_dict[func_series.name] = {f"'cat_cols': {i}, 'num_cols': {j}, 'func': {k}"}
            
            # Если же тип столбца для группировки числовой:
            else:            
                # Цикл по функциям.
                for k in list_of_agg_func:
                    # И дальше всё то же самое. 
                    # Сгруппировали, переименовали, записали в словари.
                    func_series = main_df.groupby(i)[j].transform(k)
                    func_series.name = 'strong_column_' + str(len(func_dict))
                    
                    func_dict[func_series.name] = func_series.to_dict()
                    annot_dict[func_series.name] = {f"'cat_cols': {i}, 'num_cols': {j}, 'func': {k}"}
    
    # Формируем датафрейм по основному словарю.
    func_df = pd.DataFrame(func_dict)
    # Отсеиваем лишние столбцы, согласно условию, что число теста будет больше заданного нами, что определяет силу чтолбца.
    func_df = func_df[func_df.columns[pd.Series(f_classif(func_df, main_df[targ_col].astype(int))[0]) > threshold_number]]
    # Добавляем описание в последнюю строку из словаря annot_dict.
    # Сначала формируем дф по нему, решейпим и конкатим.
    func_df = pd.concat([func_df, pd.Series(annot_dict).to_frame().T], axis=0, join='inner').reset_index(drop=True)
        
    return func_df

# Пример работы:
#list_of_cat_cols = [['hotel_town', 'year_of_review', 'quarter_of_review', 'hotel_name'], ['hotel_town', 'year_of_review', 'hotel_name']]
#list_of_num_cols = ['reviewer_score']
#list_of_agg_func = ['mean']
#targ_col = 'reviewer_score'
#
#strong_cols_auto_searcher(booking_df, list_of_cat_cols=list_of_cat_cols, list_of_num_cols=list_of_num_cols, list_of_agg_func=list_of_agg_func, targ_col=targ_col)

[to contents](#contents)

<h3 id='latlongdist'>Функция, которая вычисляет расстояние между двумя точками lattitude и longitude.</h3>

In [ ]:
def lat_long_dist(matrix, rad=6.371*1e+6, azimut=False):
    """function to calculate distance between 2 points lat and long
       https://gis-lab.info/qa/great-circles.html

    Args:
        matrix (list, array, or df): any data sequence u like
        rad (float or int, optional): radius of the earth. Defaults to 6.371*1e+6 in meters!!!.
        azimut (bool, optional): the angle in decimal degrees. Defaults to False.

    Returns:
        array: the array can be shape of (m,) or (m, n) depending on what we have submitted to the input
    """
    
    import numpy as np
    import pandas as pd
    
    if isinstance(matrix, list):
        matrix = np.array(matrix)
    elif isinstance(matrix, pd.DataFrame):
        matrix = matrix.to_numpy()
    
    # the order of sequence can be changed here
    # the order of start-end will only affect the azimuth
    # cuz for the distance there is no matter what lat or long, its the distance between 2 points
    if len(matrix.shape) == 2:
        lat_start, long_start, lat_end, long_end = matrix[:, 0], matrix[:, 1], matrix[:, 2], matrix[:, 3]
    else:
        lat_start, long_start, lat_end, long_end = matrix[0], matrix[1], matrix[2], matrix[3]
    
    # radians of lat
    f1 = lat_start * (np.pi / 180)
    f2 = lat_end * (np.pi / 180)
    
    # delta radians
    delta_lat = (lat_end - lat_start) * (np.pi / 180)
    delta_long = (long_end - long_start) * (np.pi / 180)
    
    # Haversine formula 
    a = np.power(np.sin(delta_lat / 2), 2) + np.cos(f1) * np.cos(f2) * np.power(np.sin(delta_long / 2), 2)
    haversine = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    dist = rad * haversine
    
    if not azimut:
        return dist
    else:
        # teta formula for azimut
        teta = np.arctan2(np.sin(delta_long) * np.cos(f2), np.cos(f1) * np.sin(f2) - np.sin(f1) * np.cos(f2) * np.cos(delta_long))
        
        return np.array([dist, (teta * (180 / np.pi) + 360) % 360]).T # % 360 == module 360

[to contents](#contents)

<h3 id='combinationsgenerator'>Функция для генерации комбинаций элементов входящего списка.</h3>

In [1]:
def generate_all_combinations(incoming_list, critical_num=1):
    """generate_all_combinations 

    Функция принимает на вход список и возвращает список скомбинированных списков.
    Например:
    вход ['a', 'b', 'c'] -> выход [['a'], ['b'], ['c'], ['a', 'b'], ['a', 'c'], ['b', 'c'], ['a', 'b', 'c']]

    Args:
        incoming_list (list): incoming list of something.
        critical_num (int, optional): thats if we dont need all combinations, just first half. Defaults to 1.
        
    Returns:
        list: list with all combinations of items.
    """
    
    # Внутренняя функция рекурсивная и делает она следующее:
    # На каждый length, например пример с ['a', 'b', 'c'] на входе:
    # При length=1 на выходе получаем [['a'], ['b'], ['c']]
    # При length=2 - [['a', 'b'], ['a', 'c'], ['b', 'c']]
    # При length=3 - [['a', 'b', 'c']]
    def generate_combinations(incoming_list, length):
        if length == 1:
            return [[item] for item in incoming_list]
        else:
            combinations = []
            for i, item in enumerate(incoming_list):
                for next_combination in generate_combinations(incoming_list[i+1:], length - 1):
                    combinations.append([item] + next_combination)
            return combinations
    
    # И ниже мы это всё безобразие записываем в выходящий список.
    # А критикал нум нам нужен для того, чтобы если вдруг нам не надо все все комбинации, а, например, половину...
    all_combinations = []
    for length in range(1, len(incoming_list) // critical_num + 1):
        all_combinations.extend(generate_combinations(incoming_list, length))
    return all_combinations

[to contents](#contents)

<h3 id='highcorreraser'>Функция для удаления сильно скоррелированных столбцов. (тестовая)</h3>

In [ ]:
def high_corr_eraser(df, upper_bound=0.9, lower_bound=-0.9, thresh=1, method='pearson'):
    
    func_df = df.copy()
    
    corr_df = func_df.corr(numeric_only=True, method=method)
    
    list_of_low_corr_columns_name = corr_df.corr(numeric_only=True)[(corr_df.corr(numeric_only=True) <= upper_bound) | ((corr_df.corr(numeric_only=True) >= lower_bound) & (corr_df.corr(numeric_only=True) <= 0))].dropna(thresh=thresh, axis=1).columns
    
    return func_df[list_of_low_corr_columns_name]

[to contents](#contents)

<h3 id='fclassifchitester'>Функция для тестирования силы столбца, относительно целевого. f_classif + chi2</h3>

In [ ]:
def f_classif_chi_tester(series, targ_col):
    """f_classif_chi_tester _summary_

    Функция.. проводит тестирование f_classifom и хи квадратом. И показывает число. Чем больше - тем лучше.
    Хи квадрат какой-то левый тест. Многомилионные показатели никак не влияют на модель. Как будто там что-то ломается.
    Но не суть. f_classif более правдивый.

    Args:
        series (pd.Series/pd.DataFrame): incoming series num only. Or if its df
        targ_col (pd.Series): looks like your_df['your_targ_col']

    Returns:
        _type_: _description_
    """
    import pandas as pd
    from sklearn.feature_selection import chi2 # хи-квадрат
    from sklearn.feature_selection import f_classif # anovav
    
    func_series_or_df = series.copy()
    
    # if series
    if isinstance(func_series_or_df, pd.Series):
        chi = pd.Series(chi2(func_series_or_df.to_frame().abs(), targ_col.astype(int))[0], index=['chi'])
        f = pd.Series(f_classif(func_series_or_df.to_frame(), targ_col.astype(int))[0], index=['f'])
    
    else: # if df
        if targ_col.name in func_series_or_df.columns: # drop targ_col if it exists.
            func_series_or_df.drop(columns=targ_col.name, inplace=True)
        chi = pd.Series(chi2(func_series_or_df.abs(), targ_col.astype(int))[0], index=func_series_or_df.columns, name='chi')
        f = pd.Series(f_classif(func_series_or_df, targ_col.astype(int))[0], index=func_series_or_df.columns, name='f_classif')
        
        return pd.DataFrame([f, chi]).T
    return pd.concat([f, chi])

[to contents](#contents)

<h3 id='specialgrouper'>Группировщик с применением ко всему датафрейму</h3>

In [ ]:
def special_grouper(df, bywhat, what, list_of_agg_func=['sum', 'mean', 'count', 'nunique']):
    """special_grouper 

    Согласитесь, бесит, когда надо писать несколько строчек, чтобы группировка применилась ко всей таблице.
    А метод transform принимает только 1 аргумент и ему плевать список там или нет. Хотя в документации там может быть список.
    Этот момент я до конца не понял.
    
    Эта функция - исключительно для визуальной оценки, а потом мы делаем нужный нам сериес с выбранной агрегацией через transform.
    Т.к. трансформ крутая штука на самом деле, туда можно пихнуть лямбду.
    
    Хотя и отсюда можно взять столбец, но через .loc/iloc, потому что функция возвращает тип датафрейм.

    Args:
        df (pd.DataFrame): any df.
        bywhat (str or list): name or names of columns which we want to group.
        what (str): name of col we wanna group bywhat (my logic works only for single col cuz .iloc[bla bla] (can be expanded))
        list_of_agg_func (list, optional): list of agg func. Defaults to ['sum', 'mean', 'count', 'nunique'].

    Returns:
        pd.DataFrame: a column or columns with grouping applied to the entire df as a df.
    """
    return pd.merge(df[bywhat], df.groupby(bywhat)[what].agg(list_of_agg_func).reset_index(), on=bywhat, how='left').iloc[:, -len(list_of_agg_func):]

[to contents](#contents)

<h3 id='goodbadwordssearcher'>Функция для определения положительных и отрицательных слов в предложении</h3>

In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

# Функция для определения тональности слова
def extract_sentiment_words(sentence, positive_words, negative_words):
    
    def get_word_sentiment(word, positive_words, negative_words):
        # Создание синтаксических представлений для положительных и отрицательных слов    
        positive_synsets = [j for i in positive_words for j in wordnet.synsets(i, pos=wordnet.ADJ)]
        negative_synsets = [j for i in negative_words for j in wordnet.synsets(i, pos=wordnet.ADJ)]
        
        # Определение синонимов слова в WordNet
        synsets = wordnet.synsets(word, pos=wordnet.ADJ)
        if synsets:
            # Проверка синонимов на положительность или отрицательность
            for synset in synsets:
                if synset in positive_synsets:
                    return 'positive'
                elif synset in negative_synsets:
                    return 'negative'
        # Если не удалось определить тональность
        return 'neutral'
    
    positive_list = []
    negative_list = []
    words = nltk.word_tokenize(sentence)
    for word in words:
        sentiment = get_word_sentiment(word, positive_words, negative_words)
        if sentiment == 'positive':
            positive_list.append(word)
        elif sentiment == 'negative':
            negative_list.append(word)
    return positive_list, negative_list

sentense = 'i ok feel very conflicted about this hotel unfortunately ultimately i wouldn t recommend it because with our experience the bad outweighed the good the fire alarm went off late on sunday night 11th july on the first night we stayed so naturally everyone evacuated and stood outside it was absolutely pouring so not the best start eventually a member of hotel staff came to the front desk and the alarm was silenced i would say the alarm had been going for about 10 minutes at this point though and there wasn t a staff member to tell us if it was a false alarm or real or anything she explained in a really aggressive manner that there was a fault with the alarm and heavy rain sometimes set it off she seemed irritated we had bothered to evacuate and said that it was not the first time it had happened no apology or further explanation of a fix or anything fair enough though it was close to 1am and we were all a bit moody we arrived back at the hotel on our first day and the girl at the front desk informed us the lift was broken we were on the top floor which was a pain but being able bodied it was hardly a big deal however it was not fixed for the full 4 days of our stay again no apology offered at any point they did however offer to help getting luggage downstairs which was a godsend no idea what they would have done if they had disabled guests all in all the staff were unpleasant to deal with when we returned to the hotel in the evening we arrived home late from disneyland on tuesday night and asked for some milk sachets for the room and we were told no milk is available by an exasperated girl on the front desk who was busy making a personal call we walked up the four flights of stairs and found some milk sachets on a cleaning trolley on the landing so i think she just had no interest in being helpful also just a word of advice our room 405 had the wc seperate then a shower in the middle of the bedroom'

# Определение положительных и отрицательных слов
positive_words = ['good', 'great', 'excellent', 'ok', 'awesome']  # Положительные слова
negative_words = ['bad', 'poor', 'awful', 'conflict']  # Отрицательные слова

print(extract_sentiment_words(sentence=sentense, positive_words=positive_words, negative_words=negative_words))
print(wordnet.synsets('ok', pos=wordnet.ADJ))

[to contents](#contents)

<h3 id='printmetrics'>Функция для подсчёта любых метрик на выбор, как для и просто тест выборки, так и трейн + тест выборок</h3>

In [ ]:
def print_metrics(y_test, y_test_pred, list_of_metrics, y_train=None, y_train_pred=None, names=False, metric_params_dict=False):
    """print_metrics 

    function for print any metric u like.
    P.S. some metrics have hard names, so need to adjust names in 'block to adjust names' (see code)

    Args:
        y_test (pd.Series or numpy.ndarray): y_test
        y_test_pred (list): y_test_pred (lgr.predict(X_test))
        list_of_metrics (list): list of any metrics !!CAUTION!! Metrics should be imported like: from sklearn.metrics import accuracy_score
        y_train (pd.Series or numpy.ndarray, optional): y_train. Defaults to None.
        y_train_pred (list, optional): y_train_pred. Defaults to None.
        names (list, optional): list of algo names (for ex: ['lgr_1', 'lgr_2']). Defaults to False.
        metric_params_dict (dict, optional): dict look like {'metric_name': {'parameter': 'value'}}. Defaults to False.

    Returns:
        pd.DataFrame: with our metrics
        
    ex of using:
    
    list_of_metrics = ['accuracy_score', 'f1_score', 'precision_score', 'recall_score']
    metric_params_dict = {'accuracy_score': {'normalize': False}}
    
    print_metrics(y_test=y_test, 
                  y_test_pred=[y_test_pred], 
                  list_of_metrics=list_of_metrics,
                  names=['lgr_1'], metric_params_dict=metric_params_dict)
    """
    
    import pandas as pd
    
    # if we didnt set names it will be 0, 1 etc.
    if not names:
        names = [f'{i}' for i in range(len(y_test_pred))]
    
    ################### <----- block to adjust metrics params
    # if we dont want to specify additional metric params
    if not metric_params_dict:
        metric_params_dict_func = {i:{} for i in list_of_metrics}
    # else we make empty dict with each metric and update it with needed external dict
    else:
        metric_params_dict_func = {i:{} for i in list_of_metrics}
        metric_params_dict_func.update(metric_params_dict)
    
    ################### <----- block to adjust names 
    # This for correct metrics names.
    func_list_of_metrics = []
    
    for i in list_of_metrics:
        # For ex: mean_absolute_error = MAE (if len(['mean', 'absolute', 'error']) > 2)
        if len(i.split('_')) > 2:
            func_list_of_metrics.append(''.join([j[0].capitalize() for j in i.split('_')]))
        # else r2_score = r2
        else:
            func_list_of_metrics.append(i.split('_')[0])
    ###################
    
    train_dict = {}
    test_dict = {}
    train_list = [[] for i in range(len(list_of_metrics))]
    test_list = [[] for i in range(len(list_of_metrics))]
    
    ###################
    # if we wanna see just test metrics:
    if y_train is None:
        for y_test_p in y_test_pred:
            for i, j in enumerate(list_of_metrics):
                # values of metrics
                test_value = globals()[j](y_test, y_test_p, **metric_params_dict_func[j])
                # adding to list
                test_list[i].append(test_value)
                
                # refresh dict
                test_dict[f"{func_list_of_metrics[i]}"] = test_list[i]
        
        func_df = pd.DataFrame(test_dict, index=names).T

    
    # if we wanna see train and test scores:    
    else:
        # for train_pred + test_pred
        for y_train_p, y_test_p in zip(y_train_pred, y_test_pred):
            for i, j in enumerate(list_of_metrics):
                # values of metrics
                train_value = globals()[j](y_train, y_train_p, **metric_params_dict_func[j])
                test_value = globals()[j](y_test, y_test_p, **metric_params_dict_func[j])
                # adding to lists
                train_list[i].append(train_value)
                test_list[i].append(test_value)
                
                # refresh dicts
                train_dict[f"{func_list_of_metrics[i]}"] = train_list[i]
                test_dict[f"{func_list_of_metrics[i]}"] = test_list[i]
        
        # making MultiIndex for columns
        columns = pd.MultiIndex.from_product([['train', 'test'], names])
        
        # Combining the data into one array for further creation of a df
        data = [train_list[i] + test_list[i] for i, metric in enumerate(func_list_of_metrics)]
        
        # Creating a df with a multi-index
        func_df = pd.DataFrame(data, index=func_list_of_metrics, columns=columns)
    
    return func_df